Here is the final scraper that we will be using to populate the database.  We will be pdf scraping since there were technical limitations scraping the course website. (likely protections from UCLA) 

For the demo purposes to avoid overloading our database, we will have around 10 majors with 10 classes each, with 4 discussion sections each, totalling 400 unique class codes.

In [1]:
#importing necessary packages
import PyPDF2
import re

In [2]:
#putting the pdf of the UCLA catalog into a string
#takes a while to run, ~1 minute depending on device
def extract_text_from_pdf(pdf_path, start_page=180, end_page=890):
    text = ""
    #opens pdf
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        start_page = max(0, start_page - 1)  
        end_page = min(end_page, len(reader.pages))
        #chooses which pages to parse through - the class section of the pdf in this case
        for page_number in range(start_page, end_page):
            page = reader.pages[page_number]
            text += page.extract_text()
    return text


#use path to course catalog pdf on your device
class_text = extract_text_from_pdf('/Users/dayfamily/Desktop/UCLA_Catalog_2023-24.pdf', start_page=180, end_page=890)


How we are going to input what we scrape into the database:

INSERT INTO classes VALUES ('course_code', 'dept_code', 'course_num', 'course_name', 'prof', 'discussion_code');

ex:  INSERT INTO classes VALUES ('CS000331A', 'COM SCI', '33', 'Introduction to Computer Organization', 'Reinman, G.D.', '1A');

So, we will need to make an array that has the following elements: course_code, dept_code, course_num, course_name, prof, and discussion_code.  Unfortunately, due to the inability to scrape the registrar, we will have to use dummy variables for prof and discussion_code, as those things will be variate by quarter and not something we can confirm off of the pdf version of the course catalog.  Additionally, assigning dept_code will be a little bit tricky.    

In [4]:
#a regular expression that looks for a specific pattern that matches how the pdf stores info about course_names and course_codes; not completely perfect, especially when it comes to things like fiat lux or research
course_pattern = r'(\b[A-Za-z]*\d+[A-Za-z]?)\.\s(.+?)\.'
course_matches = re.findall(course_pattern, class_text)



In [7]:
print(course_matches[:5]) 

[('1C', 'Heritage and Values'), ('19', 'Fiat Lux Freshman Seminars'), ('20C', 'Team and Leadership Fundamentals'), ('130C', 'Air Force Leadership Studies'), ('197', 'Individual Studies in Aerospace Studies')]


In [5]:
#preliminary array using what info we got so far
courses_array = []
for match in course_matches:
    course_code = match[0]
    course_name = match[1]
    courses_array.append(["", "", course_code, course_name, "", ""])


In [9]:
#prints every row - but start lower if it is crashing
for row_index, row_data in enumerate(courses_array[:5], start=1):
    print(f"Row {row_index}: {row_data}")

Row 1: ['', 'AERO ST', '1C', 'Heritage and Values', '', '']
Row 2: ['', 'AERO ST', '19', 'Fiat Lux Freshman Seminars', '', '']
Row 3: ['', 'AERO ST', '20C', 'Team and Leadership Fundamentals', '', '']
Row 4: ['', 'AERO ST', '130C', 'Air Force Leadership Studies', '', '']
Row 5: ['', 'AERO ST', '197', 'Individual Studies in Aerospace Studies', '', '']


There are over 10,000 classes, and there is no easy way to assign a major to each of them... I attempted to use different regular expressions to scrape major names on a given page and then I could subsequently assign the classes on that page to that major, but to no avail with the given formatting.  Thus, the only way to assign majors to each course, would be to do so manually, and I will not be doing all 11,000 courses.  Instead, I can do it "semi-manually", where I look at which range each class is, and assign the dept_code to those courses.  I could also write a program to do that based on perhaps if the next course number is lower than the previous one, and to change the dept_code accordingly, but that is prone to errors and having incorrect information seems worse than having incomplete information.  So, I will input dept_code information "manually" for the time being.  I will do the first couple of majors, and then prioritize popular majors like computer science, mathematics, and political science.  

In [7]:
#complete list of dept_codes, found on https://github.com/nnhien/uclacatalog/wiki
dept_abbr = {
    "AERO ST",
    "AF AMER",
    "AFRC ST",
    "AM IND",
    "ASL",
    "AN N EA",
    "ANES",
    "ANTHRO",
    "APPLING",
    "ARABIC",
    "ARCHEOL",
    "ARCH&UD",
    "ARMENIA",
    "ART",
    "ART HIS",
    "ART&ARC",
    "ARTS ED",
    "ASIAN",
    "ASIA AM",
    "ASTR",
    "A&O SCI",
    "BIOENGR",
    "BIOINFO",
    "BIOINFR",
    "BIOL CH",
    "BIOMATH",
    "BMD RES",
    "BIOSTAT",
    "C&EE ST",
    "CH ENGR",
    "CHEM",
    "CHICANO",
    "CHIN",
    "C&EE",
    "CLASSIC",
    "CLUSTER",
    "COMM",
    "CESC",
    "COM HLT",
    "COM LIT",
    "C&S BIO",
    "COM SCI",
    "CAEM",
    "DANCE",
    "DENT",
    "DESMA",
    "DGT HUM",
    "DIS STD",
    "DUTCH",
    "EPS SCI",
    "EA STDS",
    "EE BIOL",
    "ECON",
    "EDUC",
    "EC ENGR",
    "ENGR",
    "ENGL",
    "ESL",
    "ENGCOMP",
    "ENVIRON",
    "ENV HLT",
    "EPIDEM",
    "ETHNMUS",
    "FILIPNO",
    "FILM TV",
    "FOOD ST",
    "FRNCH",
    "GENDER",
    "GEOG",
    "GERMAN",
    "GRNTLGY",
    "GLB HLT",
    "GJ STDS",
    "GLBL ST",
    "GRAD PD",
    "GREEK",
    "HLT POL",
    "HEBREW",
    "HIN-URD",
    "HIST",
    "HNRS",
    "HUM GEN",
    "HNGAR",
    "IL AMER",
    "I E STD",
    "INDO",
    "INF STD",
    "I A STD",
    "INTL DV",
    "I M STD",
    "IRANIAN",
    "ISLM ST",
    "ITALIAN",
    "JAPAN",
    "JEWISH",
    "KOREA",
    "LBR STD",
    "LATIN",
    "LATN AM",
    "LAW",
    "UG-LAW",
    "LGBTQS",
    "LIFESCI",
    "LING",
    "MGMT",
    "MGMTEX",
    "MGMTFT",
    "MGMTFE",
    "MGMTGEX",
    "MGMTMFE",
    "MGMTMSA",
    "MGMTPHD",
    "MAT SCI",
    "MATH",
    "MECH&AE",
    "MED",
    "MIMG",
    "M E STD",
    "MIL SCI",
    "M PHARM",
    "MOL BIO",
    "MOL TOX",
    "MCD BIO",
    "MC&IP",
    "MUSC",
    "MSC IND",
    "MUSCLG",
    "NAV SCI",
    "NR EAST",
    "NEURBIO",
    "NEURLGY",
    "NEURO",
    "NEUROSC",
    "NEURSGY",
    "NURSING",
    "OBGYN",
    "OPTH",
    "ORL BIO",
    "ORTHPDC",
    "PATH",
    "PEDS",
    "PHILOS",
    "PHYSICS",
    "PBMED",
    "PHYSCI",
    "PHYSIOL",
    "POLSH",
    "POL SCI",
    "PORTGSE",
    "COMPTNG",
    "PSYCTRY",
    "PSYCH",
    "PUB AFF",
    "PUB HLT",
    "PUB PLC",
    "RAD ONC",
    "RELIGN",
    "ROMANIA",
    "RUSSN",
    "SCAND",
    "SCI EDU",
    "SEMITIC",
    "SRB CRO",
    "SLAVC",
    "SOC SC",
    "SOC THT",
    "SOC WLF",
    "SOC GEN",
    "SOCIOL",
    "S ASIAN",
    "SEASIAN",
    "SPAN",
    "STATS",
    "SURGERY",
    "SWAHILI",
    "THAI",
    "THEATER",
    "TURKIC",
    "UNIV ST",
    "URBN PL",
    "UROLOGY",
    "VIETMSE",
    "WL ARTS",
    "YIDDSH"
}

This is the beginning of "Semi-manually" assigning majors to classes - I will do the first 3 majors, and then the ones I specified above.  I may also fill in the remaining with "dummy major abreviations", aka fill them out with dept_codes even if they are inaccurate so that they populate the database.  

In [10]:
#I will essentially be copying this cell for as many majors as I fill out - replacing the dept_code and range accordingly.  I will be determining the range by looking at the last class listed for each major on the pdf of the general catalog.

#here is the first one: AERO ST

# Given dept_code to assign
dept_code = "AERO ST"

# Assigning the dept_code to the second column of however many classes it corresponds to
for row_index in range(0,5):
    courses_array[row_index][1] = dept_code



In [ ]:
# Printing the updated array
for row_index, row_data in enumerate(courses_array[:5], start=1):
    print(f"Row {row_index}: {row_data}")

In [12]:
# Given dept_code to assign
dept_code = "AF AMER"

# Assigning the dept_code to the second column of however many classes it corresponds to
for row_index in range(5,102):
    courses_array[row_index][1] = dept_code


In [ ]:
# Printing the updated array
for row_index, row_data in enumerate(courses_array[:5], start=1):
    print(f"Row {row_index}: {row_data}")

In [86]:
# Given dept_code to assign
dept_code = "POLI SCI"

# Assigning the dept_code to the second column of however many classes it corresponds to
for row_index in range(8997,9197):
    courses_array[row_index][1] = dept_code


In [ ]:
# Printing the updated array
for row in courses_array:
    print(row)

In [87]:
# Given dept_code to assign
dept_code = "COM SCI"

# Assigning the dept_code to the second column of however many classes it corresponds to
for row_index in range(2793,2853):
    courses_array[row_index][1] = dept_code



In [ ]:
# Printing the updated array
for row in courses_array:
    print(row)

In [88]:
# Given dept_code to assign
dept_code = "MATH"

# Assigning the dept_code to the second column of however many classes it corresponds to
for row_index in range(6975,7207):
    courses_array[row_index][1] = dept_code



In [ ]:
# Printing the updated array
for row in courses_array:
    print(row)

Okay, using the classes that I have filled out a dept_code for, I will now make a new array and fill out dummy discussion sections for them.

In [89]:
# Create a new array containing only rows with non-empty second elements
discussions_template_array = [row for row in courses_array if row[1] != ""]



In [ ]:
# Print the new array
for row in discussions_template_array:
    print(row)

In [90]:
for row in discussions_template_array:
    row[5] = "1A"



In [ ]:
for row in discussions_template_array:
    print(row)

In [94]:
discussions_updated_array = []

for row in discussions_template_array:
    # Create a duplicate row with the value of the sixth column changed to "1B"
    duplicate_row_1b = row[:]  # Create a shallow copy of the original row
    duplicate_row_1b[5] = "1B"  # Update the value of the sixth column to "1B"
    duplicate_row_1c = row[:]  # Create a shallow copy of the original row
    duplicate_row_1c[5] = "1C"  # Update the value of the sixth column to "1B"
    duplicate_row_1d = row[:]  # Create a shallow copy of the original row
    duplicate_row_1d[5] = "1D"  # Update the value of the sixth column to "1B"
    
    # Append the original row and the duplicate row to the updated array
    discussions_updated_array.append(row)
    discussions_updated_array.append(duplicate_row_1b)
    discussions_updated_array.append(duplicate_row_1c)
    discussions_updated_array.append(duplicate_row_1d)



In [ ]:
for row in discussions_updated_array:
    print(row)

Now that each class has 4 discussion sections, a dept code, a course code, and a course name, I will assign a dummy variable to be their professor - I will update it with more apt names when adding it to the website.  

In [19]:
for row in discussions_updated_array:
    row[4] = "Eggert, P.R."


In [ ]:
for row in discussions_updated_array:
    print(row)

In [66]:
dummy_profs ="""""
Abari, O.
Abbott, J.K.
Abderemane Ali, F.
Abdou, M.A.
Abe, G.J.
Abe, H.
Abel, E.D.
Aberbach, J.D.
Aberle, D.R.
Abeysekera, G.S.
Abidi, A.A.
Aboody, D.
Abou El Fadl, K.M.
Abraham, M.
Abrams, L.S.
Abramson, J.D.
Abramson, J.S.
Abrego, L.J.
Abston, J.I.
Acabado, S.B.
Ace, J.
Achesah, S.A.
Aciksoz, S.C.
Acosta Gomez, M.
Adam, B.N.
Adamian, A.M.
Adams, J.S.
Adegbola, M.B.
Adelman, H.S.
Adhanom, A.K.
Adhikari, A.
Adler, S.I.
Adney, K.J.
Adou, M.A.
Afanasyev, A.
Afifi, A.A.
Agadjanian, V.
Agadzhanyan, K.
Agak, G.W.
Agarwal, R.
Agazaryan, N.
Aggarwal, N.
Aghaloo, T.L.
Aghili, K.
Agnew, J.A.
Aharoni, D.B.
Ahitov, A.A.
Ahlberg, E.
Ahmadi, E.
Ahmadi, J.H.
Ahmadi, R.H.
Aiello, D.
Ajijola, O.A.
Akar, O.
Akcakir, G.
Akee, R.K.
Akin, I.
Akin, O.
Akotuah, F.K.
Al Rifae, K.A.
Alben, A.
Alber, F.U.
Alber, R.L.
Alberts, K.O.
Albrezzi, F.M.
Alderton, D.J.
Alexander, A.
Alexander, C.J.
Alexander, K.B.
Alexandr, Y.
Alexandrova, A.N.
Alfaro, M.E.
Alga-Sheriff, J.
Ali Ganem, K.
Ali, M.M.
Ali, N.
Alim, H.
Alkin, M.C.
Alkire, E.M.
Allakhverdyan, B.V
Allard, P.
Allen, M.J.
Allen, W.
Allison, M.A.
Allmendinger, B.
Alma, K.
Almario, J.
Almohalwas, A.M.
Alonzi, S.
Alsemiery, S.A.
Alsersawi, D.F.
Alvarado Cifuentes
Alvarado, O.I.
Alwan, A.A.
Amano, A.M.
Amano, D.
Amar, R.C.
Ambrose, R.F.
Amini, A.A.
Amlani, A.M.
Amunategui, C.
An, D.
An, J.
Anabi, J.
Anabtawi, I.
Anadol, R.
Anagnost, J.H.
Anand, S.
Anastopoulo, C.
Ancelin, H.
Andalon, R.
Anderson, C.R.
Anderson, R.D.
Anderson, S.F.
Anderson, T.L.
Anderson, Z.F.
Andre, S.E.
Andrews, A.M.
Anesi, J.T.
Angelopoulos, V.
Anheier, H.K.
Ankowski, A.N.
Annabi, N.
Anschell, J.H.
Anugom, J.C.
Aplin, T.C.
Appel, H.C.
Aprill, E.
Apter, A.H.
Arah, O.A.
Araiza, J.E.
Aralis, H.J.
Arango, J.
Arant, T.J.
Araujo, J.A.
Araujo, R.L.
Arbabzadah, N.
Arboleda, V.A.
Arciniega, E.
Ardam, J.W.
Ardell, J.L.
Arias, M.S.
Arisaka, K.
Armenta, A.
Armstrong, J.D.
Arnold, A.L.
Arnold, A.P.
Arnold, C.W.
Arnold, M.M.
Arnold, W.L.
Arranz Otaegui, I.
Arrieta Fernandez,
Arroyo Calderon, P
Arulanantham, A.
Aryal, A.
Arzeno Soltero, I.
Aschieris, J.E.
Ashby, S.
Ashton, D.J.
Askary, A.
Asker, J.W.
Aslanian, S.D.
Astor, R.A.
Asulin, R.
Ataman, B.
Atashbar, M.
Atchison, K.
Athaide, E.J.
Athavale, S.
Athreya, S.S.
Atienza, E.U.
Atienza, J.
Atkeson, A.G.
Atkinson, L.
Aurnou, J.M.
Austin, T.D.
Austin, T.R.
Avila, E.R.
Avila, J.
Avramovich, L.P.
Ayala, C.J.
Ayala, P.S.
Ayata, A.
Ayyaswamy, P.S.
Azadi, H.
Aziz, N.
Azurdia, D.E.
Babakhani, A.
Baca, J.F.
Bacher, A.D.
Bachtis, M.
Backus, K.M.
Baden, R.R.
Badger, J.B.
Baeumer, A.
Baeumler, J.
Bahk, J.
Bahtina, D.
Bahuguna, A.
Bailey, A.
Bailey, J.N.
Bailey, K.D.
Bailey, M.J.
Bain, B.R.
Bainbridge, S.M.
Baker, B.E.
Baker, B.L.
Baker, G.T.
Bakhos, C.A.
Bakhshay, S.
Balafrej, L.
Balandin, A.A.
Balderas-Fowler, R
Baldwin Clark, L.J
Balenovich, L.K.
Balkwill, S.
Ball, K.A.
Ballinger, A.E.
Balliu, B.
Balmer, P.
Balonis, M.
Bando, L.F.
Banerjee, A.S.
Banerjee, M.
Banerjee, N.
Banerjee, S.
Banerjee, U.
Banerji, A.
Baniaga, A.E.
Banner, S.A.
Baqaee, D.R.
Bar-Natan, I.
Barajas, A.
Barber, O.C.
Barber, P.H.
Barber, T.E.
Barber, V.
Barbour, M.I.
Bardacke, T.M.
Baril, J.R.
Barkan, K.R.
Barnard, H.
Barnard, R.J.
Barnes, K.R.
Barnosky, E.J.
Baron, B.M.
Barreca, A.I.
Barrera, D.S.
Barreto, M.A.
Barrett, H.C.
Barry, N.B.
Barry, P.J.
Bascara, V.
Bason, N.
Bass, J.
Bassett, J.
Bastani, R.
Bates, M.M.
Bates-Jensen, B.M.
Bathke, T.S.
Batten, C.L.
Bau, N.D.
Bauchy, M.
Bauer, D.J.
Baugh, D.A.
Baugher, Z.M.
Baum, L.G.
Bautista Durand, M
Bavi, N.
Bawden, E.A.
Bawn, K.
Baxter-King, R.G.
Baxter-Ratliff, C.
Bazmandegan, F.
Beard, J.R.
Bearden, C.E.
Becerra, R.
Becerra, Y.
Beckelhymer, S.D.
Beckett, J.S.
Beckmann, S.E.
Bedient, C.B.
Bedrossian, J.P.
Beggs, P.
Beghein, C.
Behdad, A.
Behm, J.P.
Beitz, R.W.
Beken, M.N.
Belin, T.R.
Bell, D.
Bell, E.A.
Bell, S.A.
Bell-Wilson, C.
Belnap, M.
Beltran-Sanchez, H
Belzer, D.T.
Benartzi, S.
Bendana, F.A.
Bendersky, C.B.
Bendix, K.D.
Benharash, P.
Benichou, E.
Benjamin, D.C.
Benjamin, D.J.
Bennoun, S.J.
Bensimon, D.B.
Bensinger, S.J.
Bentler, P.M.
Berbiguier, M.
Beren, C.E.
Berend, Z.
Bergero, A.J.
Berges, T.L.
Bergsneider, M.
Bergstrom, J.L.
Berk, A.J.
Berke, B.E.
Berliner, J.A.
Bern, Z.
Bernardo, A.E.
Bernstein, J.B.
Berra, J.P.
Berrios, I.K.
Berry Baker, J.N.
Berry, M.S.
Berthet, K.J.
Bertholet, A.M.
Bertozzi, A.
Berumen, A.R.
Bescher, E.P.
Besson, A.
Bhaduri, A.
Bhatia, R.
Biagas, A.H.
Bianchi, D.
Biblarz, J.P.
Bierria, A.M.
Bigham, A.W.
Bigio Luks, M.S.
Bikhchandani, S.
Bilal, M.
Bilder, R.M.
Bills, T.S.
Binder, A.M.
Binnquist, A.L.
Birch, H.E.
Bird, G.P.
Birke, R.M.
Birnbaum, S.H.
Birnholz, D.E.
Bischoff, S.A.
Biskup, M.
Bisley, J.W.
Biswal, S.
Bitan, G.
Bjekovic, N.
Bjork, E.L.
Bjork, R.A.
Black, C.V.
Black, D.L.
Blackmar, M.D.
Blackstock-Bernste
Blackwell, M.S.
Blair, G.D.
Blair, H.T.
Blaisdell, A.P.
Blake, N.T.
Blanchard, S.
Blanchard, T.O.
Blandl, F.
Blank, D.L.
Blank, I.
Blasi, J.J.
Blasius, D.M.
Blend, M.L.
Blenner, S.
Blevins, J.
Block, G.D.
Block, J.M.
Bloedel, A.W.
Blu Wakpa, T.
Blue, C.E.
Blumenberg, E.A.
Blumstein, D.T.
Board, S.A.
Bobrow, A.M.
Boccia, A.J.
Bodapati, A.V.
Bodhipak, K.
Bodiford, W.
Bodiford, W.M.
Boesel, A.
Bogdanich, C.D.
Boggs, A.
Bogumil, M.
Boj Lopez, F.E.
Boliek, E.
Bolin, M.T.
Bonavida, B.
Bondy, D.
Bonesho, C.E.
Bonk, M.
Bonta, D.M.
Bookheimer, S.Y.
Borbely, M.
Borghese, S.R.
Borman, C.J.
Borowski, K.
Borst, Z.P.
Bortnik, J.
Bosch, S.F.
Boss, M.S.
Bostani, M.
Boswell, K.S.
Bottini, N.
Bouchard, L.
Boudreaux, G.T.
Bouhaddou, M.
Bouklas, T.A.
Boum, A.
Bourgois, P.I.
Bouriakov, D.
Bouriakov, E.
Boutros, P.C.
Bower, J.E.
Bowie, J.U.
Box, A.
Boyd, W.C.
Boyle, B.
Boyle, D.E.
Bozorgnia, Y.
Bozovic, D.
Bradbury, B.D.
Bradbury, T.
Bradley, A.F.
Bradley, K.A.
Bradley, P.J.
Bragger, D.M.
Bralock, A.R.
Branco Rodakoviski
Brand, J.D.
Brand, J.E.
Brandenberg, S.J.
Brannon, T.N.
Brannum, B.A.
Branstetter, J.G.
Brantingham, P.J.
Brauer, E.R.
Braun, J.G.
Braunmuller, A.R.
Bravo Plaza, F.
Bray, A.S.
Braybrook, S.
Brecha, N.C.
Brecht, M.
Breen, E.C.
Brennan, D.A.
Brewer, T.F.
Brewster, J.
Bricker, H.L.
Brier, M.F.
Briggs, D.M.
Briggs, L.
Brillon, A.
Brinderson, L.A.
Bristow, J.E.
Brock, C.B.
Bromley, E.A.
Bronstein, J.
Bronstein, M.E.
Brook, R.H.
Brookmeyer, R.S.
Brown, C.A.
Brown, J.M.
Brown, K.A.
Brown, K.L.
Brown, M.S.
Brown, Q.
Brown, R.R.
Brown, S.
Brown, S.D.
Brown, T.A.
Brown, X.V.
Browne, D.G.
Browner, T.C.
Brozgal, L.N.
Brubaker, W.R.
Bruinsma, R.F.
Brumley, J.T.
Bryan, J.
Bryant, G.A.
Bryant, T.L.
Bucklin, R.E.
Buffett, J.B.
Bugu Buyuktas, B.
Bui, A.A.
Bui, C.V.
Bull, C.
Bulu-Taciroglu, M.
Bumford, D.
Buonomano, D.
Burge, C.T.
Burke, A.A.
Burke, J.A.
Burke, K.S.
Burkhart, S.N.
Burns, C.E.
Burns, L.M.
Burns, M.S.
Burns, R.J.
Burstein, A.T.
Burton, A.
Burton, H.
Burwick, F.L.
Buse, M.
Bush, N.
Bussel, D.J.
Bustamante, A.V.
Buswell, R.E.
Butler, S.J.
Butte, M.J.
Byock, J.L.
Byrne, T.
Cabrera-Mino, C.
Cabric, D.
Cacalano, N.A.
Cadet, W.
Cadiz, M.P.
Cadogan, M.
Caflisch, R.E.
Cagnon, C.
Cahill, C.M.
Cai, H.
Cai, L.
Caihuajia, F.
Calais, J.
Calderon, H.V.
Caldwell, A.F.
Caldwell, W.L.
Calico, J.H.
Callaghan, B.
Calzada, J.
Camacho Torres, Y.
Camacho, K.L.
Cameron, C.
Cameron, D.P.
Campbell, D.A.
Campbell, J.E.
Campbell, R.J.
Campbell, W.C.
Campion, A.
Canales Piccoli, M
Candler, R.N.
Cannon, S.C.
Cantor, R.M.
Cao, H.
Cao, M.
Cappiello, M.H.
Caram, J.R.
Caravello, J.B.
Carbado, D.W.
Carbajo, S.
Carbone, M.E.
Carcamo, J.A.
Cardon, K.N.
Cardona, G.S.
Cardwell, T.S.
Carey, M.F.
Carlin, B.I.
Carlson, A.M.
Carlucci, G.
Carman, G.P.
Carmichael, S.T.
Carnahan, D.
Carnesale, A.
Caro, F.
Carpenter, C.L.
Carpio, G.G.
Carr, D.J.
Carriero, J.P.
Carriger, M.
Carruth, A.B.
Carter, A.L.
Carter, E.F.
Carter, J.
Carter, T.A.
Cartmill, E.A.
Caruso, E.M.
Caruso, H.M.
Carvalho, I.
Casey, J.R.
Casey, T.
Caskey, J.
Castel, A.D.
Castillo Rodriguez
Castro Altamirano,
Caswell, M.
Catlin, A.R.
Cattelino, J.R.
Caudle, V.
Cavanaugh, K.C.
Cervera, F.
Cha, L.
Cha, M.
Chai, H.
Chakravarty, S.
Chamecki, M.
Champagne, D.W.
Chan, H.H.
Chandler, S.H.
Chanfreau, G.
Chang, A.J.
Chang, C.
Chang, J.P.
Chang, K.
Chang, M.
Chang, M.F.
Chang, S.
Chang, T.
Chang, V.Y.
Chang, W.
Chang, Y.
Chao, C.
Chapman, B.
Chapman, C.
Chapman, E.
Chapman, J.E.
Charles, A.C.
Charles, J.S.
Chatmon, J.C.
Chatterjee, A.K.
Chaudhuri, G.
Chavez Moreno, L.C
Chavira, D.A.
Chea, J.
Cheah, M.
Cheetham, D.T.
Chellani, T.H.
Chen, C.B.
Chen, G.
Chen, I.
Chen, J.
Chen, K.
Chen, K.J.
Chen, L.
Chen, M.S.
Chen, W.
Chen, X.
Chen, Y.
Chen, Y.Y.
Chen-Hafteck, L.
Cheng, G.
Cheng, G.C.
Cheng, P.
Cheng, Q.J.
Cheng, Y.
Chenoweth, M.D.
Chernikov, A.
Chernov, M.
Chernov, V.
Chesselet, M.
Chetverikov, D.N.
Cheung, K.
Chhay, R.M.
Chi, C.
Chiang, J.
Chiang, J.N.
Chichester, T.L.
Chief, L.
Chien, A.
Chin, A.G.
Chin, A.I.
Chiou, P.
Chism, C.N.
Chitwood, R.A.
Chiu, I.
Chkhenkeli, M.
Cho, J.
Cho, M.
Cho, M.A.
Choe, C.
Choe, R.
Choi, J.B.
Choi, J.E.
Choi, K.R.
Choi, S.E.
Chong, A.K.
Chong, E.K.
Chorpita, B.F.
Chou, T.
Chow, S.
Chowdhry, B.
Chowdhury, Z.
Christensen, A.
Christensen, D.E.
Christensen, J.A.
Christensen, T.S.
Christie, C.A.
Christodoulou, A.G
Christofides, P.D.
Christofk, H.R.
Christopoulos, J.A
Christou, N.
Chu, K.
Chu, R.
Chu, R.L.
Chui, C.O.
Chun, A.
Chun, J.J.
Chung, A.S.
Chung, G.K.
Chung, P.J.
Churchland, A.K.
Churg, E.
Chute, J.P.
Chwe, M.S.
Ciavolella, M.
Cimino, M.D.
Cisneros, N.
Ciszek, D.
Civil, G.F.
Clark, A.T.
Clark, I.E.
Clark, J.H.
Clark, L.
Clark, P.M.
Clarke, C.F.
Clarke, K.W.
Clarke, L.
Clarke, M.
Clarke, S.G.
Clausing, K.A.
Clayman, S.E.
Clayton, J.B.
Clement, S.R.
Clewett, D.
Clites, T.R.
Clubb, R.T.
Coad, G.W.
Cochran, A.J.
Cochran, S.D.
Cockrum, W.M.
Coffey, K.
Cofie, M.L.
Cohen, D.
Cohen, D.M.
Cohen, G.D.
Cohen, M.C.
Cohen, M.M.
Cohen, M.S.
Cohen, S.
Cohen, Y.
Cohn, D.H.
Colacurcio, M.J.
Colangelo, C.G.
Cole, B.L.
Cole, E.R.
Cole, S.
Coleman, A.C.
Coleman, A.L.
Coley, E.
Colicelli, J.J.
Coller, H.A.
Collett, J.L.
Collins, C.T.
Collins, D.
Collins, M.D.
Colon, E.
Colvig, C.
Colwell, C.S.
Comanor, W.S.
Commins, S.K.
Comulada, W.S.
Cong, J.J.
Conner, J.B.
Connick, P.
Convery, P.D.
Conway, S.
Cook, J.R.
Cooney, K.M.
Cooper, K.
Cooper, R.
Cooper, R.H.
Cooper, Z.D.
Cooperson, M.D.
Coppola, G.
Corbett, C.J.
Corbin, B.
Corbin-Miller, S.H
Corey, M.F.
Cornejo Salinas, G
Cornell, M.S.
Cornwell, K.W.
Coroniti, F.
Correa, S.
Cortinez, V.
Costa, D.L.
Cotton, G.
Courey, A.J.
Courtney, M.M.
Cousins, R.D.
Covarrubias, A.J.
Covington, K.L.
Cowe, P.S.
Cowen, P.D.
Cowgill, B.O.
Cox, M.
Crager, A.D.
Cramer, S.C.
Crane, R.D.
Crane, R.E.
Craske, M.
Crawley, A.K.
Crenshaw, K.W.
Crespi-Chun, C.
Cressy, M.
Critchfield, G.
Crittenden, J.W.
Crompton, J.G.
Cronin, K.H.
Crooks, G.M.
Crosbie, R.H.
Crosby, S.
Crosen, M.G.
Cross, K.A.
Cross, T.J.
Crossley, S.O.
Cruz, A.
Cruz, G.
Cruz, J.
Cubedo Saldivar, R
Cucharo, S.J.
Cuenca, E.L.
Cuevas-Collante, P
Cuff, D.
Cui, X.
Culbert, S.A.
Culik-Baird, H.
Cumberland, W.G.
Cumming, S.J.
Cunningham, C.W.
Cunningham, W.E.
Cuptor, A.M.
Curtis, J.B.
Curto, V.
Cushing, L.J.
Cychosz, M.
Czernin, J.
D'Aguiar, F.M.
D'Hoker, E.
Da Costa Alves, E.
Daboussi, L.
Dabrowska, D.M.
Dagenais, J.C.
Dago-Clark, K.J.
Dahlbom, M.
Dai, H.
Dai, X.
Dale, R.A.
Dallas, H.J.
Daly, K.M.
Damacio, B.
Damianov, A.N.
Daniel, P.M.
Danielpour, R.D.
Dante, I.
Dapretto, M.
Darabi, H.
Darby, J.T.
Darby, M.R.
Darling-Hammond, S
Darwiche, A.Y.
Dash, A.
Dave, S.
Daversa, D.
Davidson, P.L.
Davis, J.
Davis, J.D.
Davis, J.M.
Davis, M.E.
Davis, P.M.
Davoyan, A.R.
Dawson, D.W.
Day, M.D.
Dayani, E.
de Biase, L.M.
de Chadarevian, S.
de La Cruz, S.C.
de La Torre Amagua
de La Torre-Ubieta
de La Vega, I.M.
de Leon, J.P.
de Luna, M.I.
de Rezende, P.
de Robertis, E.M.
de Silva, D.L.
de Zubiaurre, M.T.
de, E.N.
Dean, M.E.
Dean, M.F.
Deb, A.
Deb, D.
Debenport, E.K.
Decan, C.L.
Decker, J.L.
Deeds, E.J.
Deguzman, J.C.
Dehoratius, N.
Deifel, N.
Deignan, J.L.
Delafield, N.L.
Dell'Angelica, E.C
Delmas, M.A.
Deloughrey, E.
Demer, J.L.
Demer, L.L.
Deming, T.J.
Demko, C.M.
Demman, B.L.
Denari, N.M.
Deng, M.C.
Deng, S.X.
Denie Higney, L.M.
Denning, J.M.
Dennis, C.M.
Depinna, N.
Deppman, F.A.
Derby, L.
Derian, S.K.
Desai, H.N.
Deshazo, J.
Desjardins, R.
Desneux, J.D.
Desveaux, J.A.
Detels, R.
Deters, K.D.
Deuel, N.A.
Deutsch, H.E.
Deverux, R.E.
Devito, D.S.
Devoe, S.E.
Devon, H.A.
DeWitt, J.
Dhaliwal, G.S.
Dhir, V.K.
Di Carlo, D.
Di Gaetano, K.
Di Rienzo, D.
Diaconescu, P.L.
Diaz, E.
Diaz, J.
Diaz, L.
Dicker, R.
Dickerson, R.E.
Dickey, D.
Dickey, S.J.
Dienstag, J.F.
Diggavi, S.N.
Dimond, T.A.
Dimuro, J.A.
Dincer, M.
Dines, J.P.
Ding, C.
Ding, H.
Ding, L.
Dipoppa, M.
Dippel, C.
Dipple, K.M.
Distefano, J.
Divakaruni, A.S.
Dixon, E.L.
Dixon, L.M.
Dixton, J.
Dizani, M.
Dluzniewski, P.
Do, K.
Do, T.H.
Dobalian, A.
Doering, L.V.
Dolbier, S.
Doleac, B.G.
Dolecek, L.
Dollase, W.A.
Dolovich, S.
Domingo, N.P.
Dominguez, E.N.
Donahue, T.R.
Donald, R.
Dong, H.
Dong, S.B.
Donlea, J.M.
Dorian, A.
Dorio, J.
Dorrani, N.
Dorsey, K.
Dorshkind, K.A.
Douglas, R.F.
Douridas, L.S.
Dover, D.J.
Doyle, A.G.
Drake, T.A.
Drew, J.B.
Drolet Rossi, A.
Drozdowski, M.R.
Drucker, J.R.
Du, H.
Du, T.
Duan, X.
Dubelman, D.
Dubinett, S.M.
Dudley, M.A.
Due, T.P.
Dumitrescu, T.
Dunham, L.R.
Dunitz, J.G.
Dunkel Schetter, C
Dunlap, M.
Dunlap, S.L.
Dunn, B.S.
Dunn, J.C.
Dunn, J.E.
Dunnavant, J.P.
Duran Zarate, C.A.
Duran, C.
Duranti, A.
Durkin, D.
Durmus, Z.
Duthie, N.N.
Duthie, T.
Dutson, E.P.
Dutton, G.E.
Duval, E.
Eagan, K.
Eagle, R.A.
Eagly, I.
Eastwood, J.O.
Ebert, M.
Ebramzadeh, E.
Economou, D.
Edgerton, V.R.
Edgren, S.J.
Edmonds, A.T.
Edu, U.F.
Edwards, S.
Edwards, T.S.
Effros, R.B.
Egea, P.F.
Eggert, P.R.
Eghbali, M.
Eidsheim, N.
Eischens, B.J.
Eisenberg, D.
Eisenberg, D.S.
Eisenberger, N.I.
Eisenman, D.P.
Eisfeldt, A.L.
Eisler, C.N.
Elashoff, D.
Elashoff, R.M.
Eldredge, J.D.
Elimam, S.
Ellingson, B.M.
Elliott, B.A.
Elliott-Newton, D.
Ellis, H.
Ellis, R.M.
Elman, R.S.
Elmore, J.G.
Emaminejad, S.
Emenaker, R.A.
Emeruwa, I.
Emigh, R.J.
Emmerich, M.D.
Enakoutsa, K.
Enders, C.K.
Ensweiler, M.L.
Enwright, J.
Epps, J.L.
Erickson, C.L.
Erickson, J.
Ernest, M.A.
Ernst, J.
Erskine, L.S.
Ertman, T.
Escarce, J.J.
Escobedo, C.R.
Escriva Bou, A.
Esdin, J.
Esfandiari, M.M.
Eskin, E.
Espinal, J.
Espino, V.
Esteves-Sorenson,
Estrada, L.F.
Ettner, S.L.
Etuk, V.
Evans, C.J.
Everhart, R.
Ewing, S.
Ezenwa, N.G.
Fabian, L.
Fabrick, S.B.
Fachin, J.M.
Faier, L.B.
Fain, G.L.
Fairlie, R.W.
Fajgelbaum, P.D.
Fakhoury, G.
Falbo, M.
Falce-Robinson, J.
Faliks, I.
Fallows, R.J.
Falls, L.
Falsafi, S.
Fan, C.C.
Fan, G.
Fanning, A.T.
Fanselow, M.S.
Farabee, D.J.
Faridani, T.H.
Farmer, D.T.
Farolfi, A.
Farooque, S.A.
Farrell, B.J.
Fasasi, A.
Fast Dog, L.
Faull, K.F.
Favila, C.D.
Favro, D.G.
Fazeli, M.
Feghali, Y.
Fei, Z.
Feigon, J.F.
Feinberg, R.M.
Feinerman, K.E.
Feinsinger, A.L.
Feldman, J.L.
Feldman, R.C.
Feler, L.
Felker, P.M.
Feng, Y.
Feng, Z.
Fensterstock, N.L.
Ferguson, J.L.
Ferketic, D.
Fernandes, P.
Ferraro, N.G.
Ferrer, C.J.
Fessler, D.
Feuer, J.
Fick, C.W.
Fielding, C.L.
Fielding, J.E.
Figliomeni, E.
Fink, R.W.
Finn, J.P.
Finnerty, L.M.
Firstenberg, I.
Fischella, M.R.
Fishbein, M.C.
Fisher, K.A.
Fisher, K.G.
Fisher, M.N.
Fisher, T.S.
Fishkin, J.R.
Fiske, A.P.
Fiske, J.
Fitzgerald, M.P.
Fitzpatrick, E.A.
Flanagan Lysy, M.M
Flanagan, K.
Fleming, M.F.
Fleshman, A.L.
Fletcher, A.
Fleurima, K.
Flint, J.
Flores, Y.N.
Florin, A.E.
Fluitt, A.H.
Flynn, P.
Foaad, L.R.
Foard, A.D.
Fogel, B.L.
Fogel, G.E.
Fong, K.N.
Fong, P.M.
Ford, C.C.
Ford, C.L.
Forman, J.
Forrest, J.
Foster, B.L.
Foster, J.G.
Foster, R.F.
Foster, S.L.
Fox, C.R.
Fox, M.E.
Fox, M.M.
Fragouli, C.P.
Frake, E.
Francis, A.M.
Franco, E.
Frand, A.R.
Frane, A.V.
Frank Jr, R.G.
Franke, M.L.
Franke, T.M.
Franklin, A.S.
Franklin, C.C.
Fraser, A.
Fraser, A.M.
Frasure, L.
Frederick, H.E.
Freedman, D.
Fregoso, O.I.
Freiman, J.D.
Freimer, N.B.
Freise, A.C.
Freixes, G.
French, E.
French, S.W.
Freshman, B.
Freyinger, C.P.
Frias, C.M.
Friberg, M.A.
Fried, I.
Friedman, H.L.
Friedman-Pappas, S
Friscia, A.R.
Frisina, M.
Frons, B.S.
Froot, D.
Frye, M.A.
Fu, P.C.
Fu, R.
Fu, S.
Fuchs, B.
Fucile, M.F.
Fulginiti, C.
Fuligni, A.J.
Funk, J.E.
Furlanetto, S.R.
Furman, Y.
Furner, J.
Furtick, L.H.
Gabato, T.
Gabbert, K.J.
Gabel, K.
Gabra, M.H.
Gabriel, S.A.
Gadh, R.
Gaffney, L.M.
Gafni, E.M.
Galashin, P.
Galbraith, K.
Gale, B.L.
Gale, N.
Galic, Z.
Gallagher, I.P.
Gallagher, K.J.
Gallagher, L.
Gallagher, S.V.
Gallerani, E.M.
Gallien, T.
Gallus, J.
Galper, M.R.
Galvan, A.
Gamal, F.
Gamboa-Kroesen, J.
Gammell, C.
Gana, N.
Gandal, M.J.
Gandhi, E.L.
Gangbo, W.
Ganz, P.A.
Ganz, T.
Gao, E.X.
Gao, J.
Gao, L.
Gao, T.
Garcia Lorenzo, G.
Garcia Sanchez, I.
Garcia, A.K.
Garcia, D.G.
Garcia, D.M.
Garcia, F.
Garcia, R.
Garcia-Garibay, M.
Gardbaum, S.A.
Garden, B.D.
Gardner Treloar, J
Garfin, D.R.
Garfinkel, A.
Garg, N.K.
Garmaise, M.J.
Garnett, J.B.
Garrell, R.L.
Garrison, N.A.
Garrison, N.J.
Garro, L.C.
Garry, A.
Garud, N.
Garza, D.R.
Gasigitamrong, J.
Gaspar de Alba, A.
Gatica, J.
Gatti, R.A.
Gavridou, S.
Gaynes, J.A.
Ge, W.
Gebremichael, M.
Geddes, B.
Gee, G.
Geerolf, F.
Geiger, J.L.
Geiogamah, H.L.
Geis, G.T.
Geisler, E.
Gekelman, W.N.
Gelbart, W.M.
Gelberg, L.
Gelmini, G.B.
Gelvin, J.L.
Genao, J.
George, S.B.
Georgis, T.P.
Gere, D.H.
Gerstel, S.E.
Geschwind, D.H.
Gettig, K.A.
Gevers, J.
Gevorkian, A.
Ghafarian, M.
Gharesi Fard, B.
Ghervas, S.
Ghez, A.M.
Ghoniem, N.M.
Ghosh, K.
Giamarino, C.D.
Gibbons, S.L.
Gibbs, A.B.
Giesler, R.
Gil Zubaran Zandom
Gilani, A.
Gilens, M.I.
Gill, M.A.
Gillespie, M.
Gillespie, R.T.
Gillespie, T.W.
Gilliland, A.J.
Gilmore, T.J.
Gim, Y.
Gimenez, I.
Gimzewski, J.K.
Giorcelli, M.
Giovannini, M.
Gipson, J.
Girard, P.E.
Giuliano, A.C.
Giuliano, P.
Givvin, K.B.
Giza, C.C.
Gizlice, A.K.
Gladkov, N.
Glanzman, D.L.
Glasgow, B.J.
Glasner, S.V.
Gleizer, O.
Glenn, B.A.
Glik, D.C.
Gober, J.W.
Godwin, H.
Goebel, D.M.
Goeman, M.R.
Goff, B.S.
Gogochuri, G.
Gogoncea, M.
Goh, K.
Gold, M.A.
Goldberg, C.E.
Goldberg, J.L.
Goldberg, R.B.
Goldberg, S.M.
Golden, M.A.
Golden-Krasner, M.
Goldhaber, J.I.
Goldin, J.
Goldman, A.H.
Goldman, A.S.
Goldman, B.
Goldsmith, J.L.
Goldstein, A.
Goldstein, C.E.
Goldstein, D.M.
Goldstein, N.J.
Golshani, P.
Golub, P.R.
Gomez, C.J.
Gomez, C.R.
Gomez, K.
Gomez, L.M.
Gomez, M.
Gomez-Pinilla, F.
Gomperts, B.N.
Goncalves, F.
Goncalves, F.M.
Gondek, J.K.
Gonen, T.
Gong, S.
Gonzaga Layaoen, K
Gonzales, D.A.
Gonzales, S.E.
Gonzalez Gomez, F.
Gonzalez, A.
Gonzalez, R.M.
Gonzalez, S.I.
Good, R.W.
Goodall, G.S.
Goodman, P.D.
Goodwin, C.
Goodwin, J.E.
Goodwin, M.H.
Goodwin-White, J.M
Goorsky, M.S.
Gopalka, T.
Gorbach, P.M.
Gordon, J.S.
Gordon, M.S.
Gorlitsky, L.
Gornbein, J.A.
Goukasian, L.
Gould, R.L.
Goyal, V.K.
Goyal, Y.
Graeber, T.G.
Gragnolati, M.
Graham, S.
Grammer, J.K.
Grandhomme, W.E.
Gray, D.H.
Gray, M.N.
Gray, N.
Green, J.L.
Green, J.O.
Green, N.S.
Green, R.E.
Green, S.A.
Green, S.D.
Greenberg, G.J.
Greenberg, M.D.
Greene, M.P.
Greene, N.H.
Greene, R.E.
Greene, S.A.
Greene, S.M.
Greenfield, P.M.
Greenspan, S.G.
Gregory, C.R.
Greif, G.
Grella, C.E.
Gresbach, V.
Grether, G.F.
Griggs, D.R.
Grimley, K.A.
Grinblatt, M.S.
Grinnell, A.D.
Grisham, W.E.
Grody, W.W.
Groeling, T.J.
Grollman, M.J.
Gross, A.D.
Gross, A.J.
Gross, E.F.
Grossi, S.
Grossman, J.H.
Grover, A.
Gruner, G.
Grynberg Azicri, A
Gu, Q.
Gu, Z.
Guan, X.
Guber, H.P.
Guedon, C.
Guerin, A.C.
Guerin, J.
Guerra, L.F.
Guerrero, H.
Guha, A.
Guha, S.
Guhin, J.J.
Guillen, L.I.
Guillermo, N.A.
Guindani, M.
Gulati, T.
Gunsalus, R.P.
Guo, F.
Guo, M.
Guo, Y.
Gupta, A.
Gupta, P.
Gupta, S.
Gupta, V.
Gurdogan, H.U.
Gurumoorthy, A.
Gurval, R.A.
Gusm&#227;O-Garcia Will
Gutierrez, C.
Gutierrez, C.C.
Gutierrez, L.A.
Gutierrez, M.
Gutierrez, R.A.
Gutierrez, S.T.
Gutperle, M.
Guttentag, M.
Guy, T.J.
Guzman, J.A.
Guzman, J.J.
Gwack, Y.
Gylys, K.H.
Gzehoviak, J.
Ha, B.A.
Ha, Y.
Haaga, Q.
Haas, G.
Haas, K.
Haberland-Noce, M.
Habibi, P.B.
Hackett, M.J.
Hackmann, M.B.
Hadadi, P.
Haddad, V.P.
Hadjeris, F.
Hadjioannou, A.
Hafeznezami, S.
Haggag, K.F.
Hagigi, F.A.
Hahm, A.
Hahn, J.
Haigh, B.P.
Haigh, G.
Hajal, N.J.
Hak, R.R.
Hakak, L.
Hale, S.
Haley, S.
Halfon, N.
Hall, A.D.
Hall, C.A.
Hall, R.
Hallem, E.A.
Halligan, C.
Halperin, E.
Hamilton, A.B.
Hamilton, N.M.
Hamilton, P.
Hammersley, R.
Hammett, W.H.
Hamouzas, T.
Hamza, A.T.
Han, E.
Han, F.
Han, J.J.
Han, Y.
Hancock, H.J.
Handcock, M.S.
Hankinson, O.
Hanna, W.M.
Hanscom, C.P.
Hansen, B.M.
Hansen, C.L.
Hansen, E.
Hansen, G.D.
Hansen, M.P.
Hanson, C.R.
Hanulik, C.
Hanzalik, D.
Hao, Y.
Harawa, N.T.
Harder, S.
Harget, M.J.
Harper, R.M.
Harran, P.G.
Harrigan, R.J.
Harris, A.A.
Harris, C.I.
Harris, J.A.
Harris, J.C.
Harris, K.K.
Harris, N.G.
Harris, P.J.
Harrison, C.A.
Harrison, M.
Harrison, T.J.
Hart, L.
Hartenberger, L.A.
Hartenstein, V.
Hartman, A.R.
Hartman, C.
Hartman, E.K.
Hartman, S.E.
Hartman-Glaser, B.
Hartshorn, M.
Harvey, J.
Harwell, G.T.
Haselswerdt, E.H.
Haselton, M.G.
Hasen, R.
Hashibe, M.
Haskell, N.M.
Hassnaoui, A.
Hasson, T.W.
Hauck, J.
Hauser, J.
Hawkins, R.
Hayes, B.P.
Hayes-Bautista, D.
Hayn, C.
Hays, R.D.
Hazard, E.R.
Hazlett, C.J.
He, C.
He, H.S.
He, J.
He, L.
He, X.
He, Y.
Head, E.M.
Hearn, K.N.
Hecht, M.J.
Hecht, S.B.
Heck, J.E.
Heckerman, D.E.
Heffes, Y.
Heilemann, M.V.
Heilman, J.W.
Heinrich, A.
Heise, U.K.
Heitz, K.A.
Helton, G.
Henary, M.
Hendelman, T.
Henderson, G.
Henderson, S.A.
Hendon, G.W.
Hendry, S.J.
Henry, J.X.
Heo, K.
Herman, A.D.
Herman, B.
Herman, D.R.
Hernandez, C.
Hernandez, F.A.
Hernandez, J.
Hernandez, L.
Hernandez, L.M.
Hernandez, Y.Y.
Hernandez-Leon, R.
Herndon, M.A.
Herrera, J.C.
Herrera, V.
Herrera, W.A.
Herring, C.J.
Herschman, H.R.
Hershfield, H.E.
Herskovic, B.
Hester, E.W.
Heuveline, P.C.
Hevener, A.L.
Heymann, S.J.
Hiatt, P.J.
Hickman Haynes, L.
Hicks, M.I.
Hida, H.
Hieronymi, P.
Higbie, F.T.
Higgins, C.
Higgins, D.R.
Hildebrand, M.J.
Hill, J.D.
Hill, K.L.
Hill, M.A.
Hinman, J.D.
Hinojosa-Ojeda, R.
Hinshaw, J.B.
Hipolito, E.D.
Hirano, K.
Hirsch, A.G.
Hirsch, A.M.
Hirsh, W.T.
Hitchner, T.A.
Ho, A.Y.
Ho, E.M.
Ho, F.Y.
Ho, H.
Ho, J.H.
Ho, T.C.
Hoang, I.B.
Hodge, F.S.
Hodgson, T.E.
Hodko, D.
Hoek, E.M.
Hoff, N.A.
Hoffman, J.M.
Hoffman, L.F.
Hoffmann, A.
Hofnung, T.
Hoftman, G.
Hogan, A.
Hogan, J.T.
Hogenkamp, S.M.
Holbrook, A.J.
Holczer, K.
Holguin, G.
Hollan, D.W.
Hollenbeck, B.W.
Holloway, I.W.
Holloway, T.M.
Hollywood, J.P.
Holmbeck, E.R.
Holmes, C.M.
Holt, L.E.
Holyoak, K.
Homola, J.
Honda, K.
Hong, B.
Hong, C.
Hong, D.W.
Hong, K.
Hong, W.
Hong-Hermesdorf, A
Honka, E.
Hood, M.
Hood, M.R.
Hookasian, E.
Hoopes, A.
Hooshmand, M.J.
Hoover, K.M.
Hopenhayn, H.A.
Hopkins, A.
Hopkins, J.
Hopper, J.S.
Hornby, L.E.
Horner, B.J.
Horowitz, C.
Horvat, F.
Horvath, S.
Hossain, M.
Hotez, E.
Hou, C.
Hou, V.
Houk, K.N.
Hovda, D.A.
Hovhannisyan, T.
Howard, B.D.
Howard, T.
Hristakeva, S.
Hsiai, T.
Hsiao, C.
Hsiao, E.Y.
Hsieh, C.
Hsu, A.
Hsu, P.T.
Hsu, W.
Hu, B.
Hu, J.
Hu, J.K.
Hu, N.
Hu, S.
Hu, Y.
Hualpa, L.D.
Huang, B.K.
Huang, E.
Huang, G.J.
Huang, H.
Huang, H.Z.
Huang, J.
Huang, S.
Huang, Y.
Hubbell, W.L.
Hudson Masters, S.
Hudson, E.R.
Hudson, M.B.
Hudson, P.J.
Huehls, M.A.
Hueth, J.A.
Hugh Sam, J.M.
Hughes, J.S.
Huk, A.
Huljich, G.
Hull, C.M.
Hummel, R.F.
Hundley, R.M.
Hung, F.
Hunt, D.M.
Hunt, L.A.
Hunter, C.V.
Hunter, M.
Hunter, M.A.
Hunter, T.
Huo, Y.J.
Huppin, M.A.
Hur, T.
Hurtado, S.
Hurwitz, B.
Hussain, S.K.
Hussein, S.
Hyams, N.M.
Hyde, C.
Hyde, C.L.
Hyman, J.T.
Iacoboni, M.
Ibanez, M.
Ibrahim, J.S.
Iglehart, A.P.
Ignarro, L.J.
Ikotun, O.
Illes, L.I.
Imai, K.
Inaba, J.
Ingenito, D.
Ingersoll, G.M.
Ingersoll, R.V.
Inkelas, M.
Inlender, T.
Ino, A.S.
Iqbal, R.
Iravani, M.
Iruela-Arispe, L.M
Isagulyan, A.
Isen, J.J.
Isenberg, B.M.
Isken, J.
Issenberg, S.P.
Itskhoki, O.
Ivanova-Sullivan,
Iwamoto, K.S.
Iwasaki, S.
Iwasaki, T.
Iyer, S.S.
Izadpanah, A.
Izquierdo Edler, A
Izsak, C.
Jackman, A.P.
Jackson, A.P.
Jackson, D.D.
Jackson, N.J.
Jacobs, D.K.
Jacobs, J.P.
Jacobsen, S.E.
Jacobson, T.A.
Jacobson, V.
Jacoby, S.M.
Jacoby, V.L.
Jadallah, C.C.
Jaeger, A.T.
Jager, E.
Jakobsen, H.
Jalali, B.
James, A.E.
James, D.A.
James, I.
James, M.T.
James, S.C.
James, T.L.
Jamison, S.
Jampol, J.
Jamshidi, N.
Jang, S.
Jaquette, O.
Jaramillo, S.
Jarrahi, M.
Jarsky, K.M.
Jassby, D.
Jauregui, J.C.
Jaurretche, C.M.
Javanbakht, M.
Jay, J.A.
Jenko, F.S.
Jeon, C.Y.
Jeon, M.
Jeon, S.
Jeong, D.
Jeong, H.
Jepson, M.C.
Jepson, N.S.
Jerrett, M.L.
Jewett, A.
Jewitt, D.C.
Jiang, C.
Jiang, H.
Jiang, L.
Jin, L.
Jo, J.M.
Johanson, C.
John, V.
Johns, L.N.
Johnson, B.A.
Johnson, C.L.
Johnson, D.
Johnson, D.M.
Johnson, E.C.
Johnson, G.
Johnson, J.
Johnson, J.K.
Johnson, J.R.
Johnson, K.
Johnson, M.C.
Johnson, P.J.
Johnson, R.
Johnson, R.C.
Johnson, S.E.
Johnson, S.P.
Johnson, S.R.
Johnson, T.
Johnson, T.L.
Johnsrude, B.P.
Johnston, J.
Jonas, S.J.
Jones, F.L.
Jones, G.
Jones, J.
Jones, L.
Jones, R.M.
Joo, J.
Jordan, S.E.
Joseph Davey, D.L.
Joseph, S.J.
Joshi, C.J.
Joshi, S.H.
Jovanov, C.
Jovanovic, M.
Jrade, E.
Ju, H.
Ju, J.
Ju, S.
Juang, P.
Jugdeo, V.
Juliano, L.
Julius, A.J.
Jun, S.
Jung, H.
Jung, M.E.
Jung, O.
Jung, S.
Jung-Kim, J.J.
Jungmann, B.
Junker, N.M.
Juvonen, J.H.
Kadambi, A.
Kady, A.
Kaganovsky, L.
Kagawa Singer, M.
Kahl, M.
Kahng, H.J.
Kaiser, W.J.
Kakoulli, I.
Kalofonos, I.A.
Kalsi, H.K.
Kamaha, A.
Kamajian, N.E.
Kamal Idrissi, E.
Kamariza, M.
Kamei, D.T.
Kandamby, K.R.
Kane, T.L.
Kaneko, N.
Kaner, R.B.
Kaneyasu, M.
Kang, B.
Kang, G.
Kang, J.
Kang, M.K.
Kang, Z.
Kannan, S.
Kantner, B.R.
Kantor, E.
Kao, J.C.
Kao, J.L.
Kao, R.Q.
Kaplan, D.
Kaplan, D.B.
Kaplan, I.R.
Kaplan, M.S.
Kapur, A.A.
Karagozian, A.R.
Karanicolas, M.
Karcher, R.T.
Kareem, S.T.
Kareiva, P.M.
Karimli, L.
Karlan, M.S.
Karlsgodt, K.H.
Karlsson, E.J.
Karmarkar, U.S.
Karney, B.R.
Karr, S.F.
Karvonides Dushenk
Kasari, C.L.
Kashanchi, R.
Kashefi, A.
Kasko, A.M.
Kassab, L.
Kassis, A.
Kataoka Endo, S.H.
Kaufman, E.K.
Kaur, S.S.
Kaushal, A.
Kavehpour, H.
Kavner, A.
Kawanishi, Y.
Kay, A.C.
Kay, K.A.
Kazanci, A.
Kazaras, P.D.
Kazmi, S.
Keany, L.
Keating, L.
Keating, P.
Keel, T.D.
Keenan, E.L.
Kehoe, E.
Keine, S.
Keller, E.J.
Kelley, J.C.
Kelley, R.D.
Kellman, P.
Kellner, D.M.
Kelly, A.
Kelly, H.A.
Kelly, K.A.
Kelly, R.M.
Kelty, C.M.
Kendall, R.K.
Kennedy, P.J.
Kennelly, K.M.
Kennison, R.L.
Keranen, J.P.
Kerfeld, C.A.
Kern, A.N.
Kernell, G.C.
Kerr, A.Z.
Kersey, K.W.
Kersey, S.E.
Ketai, R.L.
Keum, B.
Keyes, C.L.
Khademhosseini, A.
Khakh, B.S.
Khalid Jawed, M.
Khalifa, K.E.
Khan, A.
Khan, A.M.
Khan, Z.A.
Khankan, R.R.
Khare, C.
Kheifets, L.I.
Kheradyar, A.T.
Khimasia, A.
Khitrik, M.
Khoja, A.M.
Kicenski, K.K.
Killip, R.
Kim, A.H.
Kim, A.M.
Kim, C.
Kim, D.
Kim, E.
Kim, G.
Kim, G.J.
Kim, H.
Kim, J.
Kim, J.J.
Kim, K.Y.
Kim, L.
Kim, M.
Kim, M.P.
Kim, R.
Kim, R.H.
Kim, S.
Kim, W.
Kim, Y.
Kim-Farley, R.J.
Kimgina, J.
Kinberg, M.H.
Kinderman, W.A.
Kindred, H.A.
King, H.R.
Kipen, D.M.
Kirk, J.C.
Kirk, R.
Kirk, S.A.
Kirkland, B.
Kirkman, T.D.
Kirsch, V.H.
Kistner, K.M.
Kite, B.E.
Kivelson, M.G.
Klein, G.L.
Kleinhesselink, A.
Kleinrock, L.
Kligman, G.
Kligman, M.L.
Kline, N.G.
Klokkevold, P.
Klomhaus, A.
Klyman, R.
Knapp, R.L.
Knowlton, B.
Ko, J.
Ko, S.
Kochis, R.
Kodambaka, S.
Koeffler, H.P.
Koehler, C.M.
Koerner-Al-Rawi, J
Kogar, A.
Koh, C.
Kohn, D.B.
Kok, J.F.
Koltsova, E.
Komar, K.L.
Kominski, G.F.
Kondrath, R.R.
Kong, M.H.
Kong, Y.
Koniak-Griffin, D.
Konstantopoulos, G
Koo, S.
Koopman, H.
Koral, M.R.
Korb, A.
Korf, R.E.
Koripalli, A.
Korkman, Z.K.
Kornblum, H.I.
Korobkin, S.A.
Koropeckyj, R.
Koser Schwartz, B.
Koslov, E.C.
Kosuri, S.
Kou, Y.
Koulloukian, N.L.
Kouloujian, H.
Koury, J.
Kovacova, G.
Kowalski, M.
Koya, P.D.
Koziol, M.J.
Kraft, H.H.
Kraft, N.
Krajbich, I.M.
Krakow, D.
Kramer, T.D.
Kramer, T.J.
Krantz, D.E.
Krasne, F.
Krasne, S.J.
Kratzer, C.C.
Kraus, P.J.
Krause, N.
Kraut, J.
Kreide, A.T.
Kreiman, J.E.
Kreiss, E.
Kremer, C.T.
Kremer-Sadlik, T.
Krems, J.A.
Kresin, S.C.
Krishna, A.
Kristal, E.
Kroskrity, P.V.
Krouse, I.
Kruglyak, L.
Krutowski, R.
Kubzansky, J.R.
Kudyma, A.
Kuhl, C.B.
Kuhn, R.
Kuklin, J.P.
Kulriya, M.
Kumar, A.
Kumar, N.T.
Kumar, R.
Kundu, S.
Kuney, A.K.
Kung, K.C.
Kuntz, T.
Kuo, A.A.
Kuo, C.Y.
Kuo, M.
Kuo, T.Y.
Kurdistani, S.K.
Kurtz, W.
Kusenko, A.
Kwok, I.Y.
Kwon, M.
Kwon, O.
Labran, R.E.
Lacayo, C.O.
Lai, A.
Lajonchere, C.M.
Lake, F.L.
Lake, J.A.
Lake, W.
Lal, V.
Lamb, J.M.
Lamberson, P.J.
Lamparelli, A.
Lan, X.
Landaw, E.M.
Landecker, H.L.
Landers, E.M.
Landis, D.N.
Lang, C.M.
Lang, K.S.
Langdon, J.S.
Lange, K.L.
Langer, M.
Lannan, R.
Lanning, D.P.
Laredo, J.D.
Larison, B.J.
Larkin, I.I.
Larkin, J.E.
Larkins, S.A.
Larson, D.W.
Lasch, J.G.
Lashgari, M.
Laski, F.A.
Lau, C.H.
Lau, S.
Lavagnino, C.G.
Lavelle, L.
Lavin, S.
Lavine, A.G.
Lavretsky, H.
Law, A.
Lawless, C.M.
Lawrence, B.S.
Lawrence, G.
Lawson, M.
Lazar, J.
Lazarus, B.D.
Lazazzera, B.A.
Le Belle, J.
Le Guin, E.C.
Le, M.K.
Le, T.M.
Leaf, D.A.
Leamer, E.E.
Leap, J.J.
Leazer, G.H.
Lebeau, R.T.
Lechner, M.G.
Ledoit, O.R.
Lee, A.
Lee, B.
Lee, C.
Lee, C.J.
Lee, D.
Lee, D.O.
Lee, E.
Lee, H.
Lee, J.
Lee, J.J.
Lee, J.S.
Lee, K.
Lee, K.I.
Lee, M.
Lee, M.L.
Lee, N.
Lee, P.
Lee, R.
Lee, R.C.
Lee, S.
Lee, S.K.
Lee, S.P.
Lee, S.S.
Lee, Y.
Lefkowitz, D.S.
Leflore, G.
Lehman, K.
Lehtman, N.V.
Leibrandt, D.R.
Leitch, D.B.
Lejko-Lacan, V.
Lekovic, E.
Lelmi, J.
Lemelson, R.B.
Lenartowicz, A.
Lenet, S.H.
Lenhoff, G.D.
Lens, M.C.
Lent, J.D.
Lesko, L.N.
Lesnick, J.
Lesure, J.
Lesure, R.G.
Lettenmaier, D.P.
Letz, J.
Leuchter, A.F.
Levandofsky, J.
Leveneur, F.
Levenson, A.P.
Levenson, L.
Levi, D.S.
Levin, A.
Levin, W.C.
Levine, A.J.
Levine, N.
Levine, R.D.
Levinson, D.J.
Levitt, J.
Levitz, T.J.
Levy, B.D.
Levy, S.R.
Levy-Storms, L.F.
Lew, M.E.
Lew, V.
Lewis, B.
Lewis, J.B.
Lewis, M.A.
Li, A.
Li, C.
Li, D.
Li, G.
Li, H.
Li, J.
Li, K.
Li, L.
Li, M.
Li, N.
Li, Q.
Li, S.
Li, X.
Li, Y.
Li, Z.
Liang, L.
Liao, C.
Liao, J.C.
Liao, Z.
Librandi, D.
Lichtle, J.
Lichtman, D.G.
Lieberman, M.B.
Lieberman, M.D.
Liebing, H.
Lieder, F.
Lievrouw, L.A.
Lifland, A.M.
Light, J.A.
Lim, Z.
Lin, C.
Lin, C.C.
Lin, C.X.
Lin, L.
Lin, L.Y.
Lin, N.
Lin, S.
Lin, Y.
Lindberg, L.E.
Lindemann, J.H.
Ling, J.
Linsley, C.
Lipman, A.J.
Lippit, S.M.
Lipps, J.R.
Lipps, M.D.
Lipshutz, G.S.
Lithgow-Bertelloni
Littenberg, S.M.
Little, A.L.
Litvinsky, M.L.
Litwin, M.S.
Liu, C.
Liu, G.
Liu, H.
Liu, H.Y.
Liu, J.M.
Liu, K.
Liu, S.
Liu, W.
Liu, X.
Liu, Y.
Liu, Z.
Lizardo, O.A.
Llerandi, M.
Lleras-Muney, A.
Lloyd-Smith, J.O.
Lo, R.S.
Lochstoer, L.A.
Locke Suchodolski,
Locke, A.
Lofchie, M.F.
Loh, K.C.
Lohmann, A.S.
Lohmann, S.
Lohmueller, K.E.
London, E.D.
Londono-Velez, J.
Long, E.F.
Long, J.A.
Longcore, T.R.
Longstaff, F.A.
Loo, J.A.
Looby, C.J.
Lopez Droguett, E.
Lopez Gaspar de Al
Lopez, A.I.
Lopez, B.T.
Lopez, E.F.
Lopez, I.
Lopez, I.A.
Lopez, J.G.
Lopez, M.
Lor, S.
Loria, I.
Losada Olmos, I.M.
Lotfollahi, S.S.
Lough-Stevens, M.J
Loui, M.C.
Louie, J.
Loukaitou-Sideris,
Loukota Sanclement
Love-Bibbero, L.A.
Low, D.A.
Lowry, W.E.
Loya, J.C.
Loyd, H.M.
Loza, S.J.
Lu, D.C.
Lu, H.
Lu, M.
Lu, Q.
Lu, S.
Lu, Y.
Lu, Z.J.
Luberto, J.
Luce, C.E.
Lucero, R.J.
Luft, A.R.
Lugo, A.E.
Lunde, A.O.
Luo, C.
Luo, J.
Luong, J.
Luong, T.Q.
Lupo, N.M.
Lupola, G.L.
Lusis, A.J.
Lux, R.
Lydon, G.E.
Lynch, C.
Lynch, J.W.
Lynch, T.G.
Lynch, T.M.
Lynn, C.R.
Lynn, G.S.
Lyon Grossman, K.A
Lyons, K.M.
Lyons, S.A.
Lysy, A.
Lytle Hernandez, K
Lyu, H.
Lyu, Y.
M'Closkey, R.T.
Ma, J.
Mabanckou, A.M.
MacDonald, G.M.
Macey, P.M.
MacFadyen, D.W.
Machuca, M.
Macias, R.F.
Maciel, F.V.
Macinko, J.A.
Mack, C.A.
Mack, C.B.
Mack, J.J.
Mackenzie, M.G.
Mackenzie-Graham,
Mackintosh, K.F.
MacLaren, H.
MacNeil, L.J.
Madeloni, L.J.
Madley, B.L.
Madrid Padilla, O.
Magana, M.R.
Mahajan, A.K.
Mahajan, G.
Mahapatra, A.
Mahendra, S.
Mahjoub, L.R.
Mai, H.
Mai, J.C.
Maida, C.A.
Maierhofer, T.J.
Maji, R.
Makdisi, S.
Mal, A.K.
Malabre, S.B.
Malamuth, N.M.
Malek, M.A.
Malik, A.A.
Maliski, S.L.
Malkan, M.A.
Malkin, I.
Malkov, S.
Malloy, T.
Mallya, S.M.
Malone, M.B.
Mamani, H.
Mamber, S.D.
Mamer, J.W.
Mamidipudi, S.V.
Mancevice, N.A.
Mangione, C.M.
Maniquis, R.M.
Mankekar, P.
Mann, D.R.
Mann, J.L.
Manning, C.E.
Manojlovic, M.
Manouelian, V.H.
Manousiouthakis, V
Mansi, R.
Manuelides, E.
Manville, M.K.
Manzano, J.R.
Maoz, U.M.
Mapes Frances, A.
Maquiling, D.M.
Marban, E.
Marchant, E.A.
Marchiori-Wong, M.
Marcot, J.D.
Marcus, D.W.
Margot, J.
Margulis, S.A.
Marian, J.
Marin, A.M.
Marinaro, P.
Marinello, M.L.
Marino, K.M.
Markarian, L.
Markham, J.E.
Markham, Y.K.
Markovic, D.
Markowitz, A.
Marks, L.S.
Marks, V.E.
Marlier, M.E.
Marmorstein, V.E.
Marotti, W.
Marquez, R.
Marquez-Garban, D.
Marshall, R.A.
Marshall, T.R.
Marsilii, W.J.
Marston, J.C.
Martelli, F.K.
Martignon, M.
Martin, E.E.
Martin, K.C.
Martinez Greenleaf
Martinez Moron, N.
Martinez, A.M.
Martinez, C.M.
Martinez, J.A.
Martinez, K.L.
Martinez, L.
Martinez, L.M.
Martinez, L.N.
Martinez, O.M.
Martinez, S.M.
Martinez, T.C.
Martinez-Fernandez
Martino, L.
Martins Lino, P.S.
Martinson, H.G.
Martis, D.
Martner, A.
Marturano, J.
Masmanidis, S.
Mason, T.G.
Massaly, N.
Masserman, M.D.
Massey, P.
Masuoka, N.R.
Matabang, S.L.
Mateu, V.
Mathews, P.J.
Mathur, S.
Matsumoto, V.J.
Mattenson, L.M.
Matthews, C.J.
Matus, E.
Matute, J.M.
Matzkin, R.L.
Mauleon, E.
Mauro, J.
Maxwell, A.E.
May, J.W.
Mayeda, E.
Maynard, H.D.
Mayorga, O.J.
Mays, K.T.
Mays, V.M.
Mazzaferro, A.M.
Mazzocco, M.
Mc Lurkin, S.M.
McAuliffe, S.P.
McBride, J.
McBrien, B.M.
McCaney, J.T.
McCann, C.F.
McCann, R.
McCarthy, L.L.
McCarthy, W.J.
McCarty, T.L.
McClelland, S.L.
McClendon, M.C.
McClure, K.M.
McCullough, B.H.
McCumber, J.A.
McDevitt, E.P.
McDonald, W.C.
McDonough, P.M.
McDowell, C.L.
McDowell, J.
McEachern, C.
McEnerney, B.
McEvoy, M.M.
McGarry, K.M.
McGovern, I.R.
McGowan, A.
McGrath, N.
McHugh, K.A.
McIver, M.L.
McJunkin, K.S.
McKay Villegas, A.
McKeegan, K.D.
McKeiver, D.T.
McKenna, M.P.
McKeown, D.F.
McKinney, R.
McKinnon, K.
McLaurin, C.N.
McLean, C.
McLean, I.S.
McMillan, U.G.
McMillian, R.T.
McMurtray, A.M.
McNamee, L.A.
McNitt-Gray, M.
McPherron, R.L.
McWilliams, C.B.
McWilliams, J.C.
Meagher, M.R.
Mechikoff, R.A.
Medellin Guerrero,
Mehdizadeh, N.
Mehringer, M.
Mehta, A.M.
Mehta, M.R.
Meikle, N.C.
Meinhausen, C.E.
Meites, N.G.
Meka, R.
Melas, P.
Melchert, H.C.
Melchor, D.A.
Meldrum, M.
Melega, W.P.
Melikyan, A.
Mellor, A.K.
Memarzadeh, S.
Mendenhall, M.A.
Mendes, L.H.
Mendez, M.F.
Mendler, A.H.
Mendler, J.M.
Mendoza, A.R.
Mendoza-Denton, N.
Meng, L.
Meng, T.
Mengual, M.
Menjivar, T.
Mentes, J.C.
Menz, G.
Meranze, M.
Mercer, C.L.
Merchant, S.
Merino-Hernandez,
Merkin, S.S.
Merkurjev, A.S.
Merlic, C.A.
Merriam, J.R.
Merrill, C.
Merritt, J.
Mesghali, F.
Messadi, D.V.
Metzger, E.M.
Metzger, S.A.
Metzner, W.H.
Meyer, A.S.
Meyer-Lorey, R.
Meyer-Ter-Vehn, M.
Meyers, E.A.
Meyers, H.D.
Meza Lazaro, Y.
Meza, J.I.
Miake-Lye, I.M.
Miao, J.
Miceli, C.B.
Miceli, M.C.
Michaeli, B.
Michaels, J.
Michailidis, G.
Michel, S.K.
Michell, K.B.
Michels, K.B.
Mikkola, H.K.
Milburn, D.S.
Milburn, N.G.
Milgram Cohen, H.
Milgrim, W.
Millard-Ball, A.S.
Miller Laszlo, C.
Miller, D.H.
Miller, G.A.
Miller, J.F.
Miller, J.H.
Miller, J.T.
Milliron, R.B.
Millstein, T.D.
Miltner, A.
Mimiaga, M.J.
Min, E.A.
Min, H.
Min, Z.
Minissian, M.B.
Minkova Stockwell,
Minor, C.F.
Minor, D.B.
Miranda, K.I.
Miranda, R.
Mirer, M.J.
Mirzaeian, N.
Mirzasoleiman, B.
Misic, V.
Mistry, R.S.
Mitchell, C.A.
Mitchell, J.L.
Mitchell, K.
Mitchell, L.M.
Mitchell, R.L.
Mithlo, N.M.
Mititelu, A.
Mitsunaga, J.I.
Mittal, P.
Mittelman, S.D.
Miyake, L.R.
Mladina, P.J.
Mo'e'hahne, H.
Modlin, R.L.
Mody, I.
Moe, M.L.
Moehnke, S.T.
Moeller, C.A.
Moges, B.
Mohanty, S.
Mohseni, H.
Moio, J.A.
Molina, C.
Molina, R.A.
Moloudi, S.
Momdjian, M.
Momeni, E.
Moment, A.
Mona, C.
Monaco, C.
Monaghan, E.J.
Monarrez, P.
Monbouquette, H.G.
Monge Larrain, J.M
Monkkonen, P.
Monroe, A.R.
Montgomery, A.
Montgomery, M.J.
Monti, M.M.
Montoya, A.K.
Montoya, R.D.
Montufar Cuartas,
Moodliar, R.
Moon, A.
Moon, S.
Moon, W.
Moorad, J.S.
Moore, A.
Moore, A.D.
Moore, A.J.
Moore, E.F.
Moore-Cantwell, C.
Moorjani, D.
Mora Laciana, C.
Moraga Saez, J.L.
Morales Guio, C.G.
Morales Macedo, S.
Morales, E.
Morales, G.J.
Morales, S.D.
Morcom, A.F.
Moreland, J.E.
Morgan, K.A.
Morgin, K.L.
Mori, W.B.
Morris, M.R.
Morris, R.J.
Morris, S.P.
Morrison, C.L.
Morrison, R.
Morrison, S.L.
Morrow, E.E.
Morrow, J.S.
Morton, I.
Moschovakis, Y.N.
Moseley, A.M.
Moshaverinia, A.
Mosleh, A.
Moss, M.K.
Motomura, H.
Mott, C.M.
Moucheraud, C.
Mouhktar, H.
Mousa, N.
Mousa, S.
Mousavi, S.
Moy, T.J.
Moya, S.
Mufti, A.R.
Muir, T.S.
Mukhija, V.
Muldoon-Hules, K.M
Mullen, H.R.
Muller, Y.M.
Mullin, M.
Mulugeta, M.
Munoz, J.
Munro, P.
Murat, L.
Murphy, J.M.
Murray, B.J.
Murray, C.A.
Murray, M.J.
Murray, S.
Murri, J.W.
Murtaugh, K.S.
Murthy, J.Y.
Muscatello, V.S.
Musumeci, P.
Mutlu Akaturk, H.
Myers, D.N.
Mynhier, L.G.
Nabokov, P.
Nachenberg, C.S.
Naclerio, M.E.
Nadimi, B.
Nadimi, S.
Nadjimzadah, A.
Naeim, A.
Nag, A.
Nagamine, Y.
Nagaoka Toyotoshi,
Nagappan, A.
Nagy, J.F.
Nagy, P.A.
Naiman, T.H.
Nair, S.E.
Nair, V.M.
Nakagawa, T.
Nakamoto, S.S.
Nakamura, E.M.
Nakano, A.
Nakaoka, S.J.
Naoz, S.
Narang, P.
Naranjo, B.B.
Narasimhan, S.
Narins, P.M.
Narr, K.L.
Narro, V.H.
Nasiali, M.A.
Nast, C.C.
Nathanson, D.A.
Natterson, B.J.
Nava, M.J.
Nava-Landeros, I.
Navarro Pope, B.A.
Navarro, K.
Nechrebecki, M.M.
Needell, D.
Needleman, J.
Neelin, J.D.
Neeman, I.
Neer, E.M.
Negrini, E.
Neil, D.G.
Neill, L.A.
Nelischer, C.S.
Nelson, H.M.
Nelson, S.F.
Nemeth, E.
Nemirow, B.
Nersessian, A.J.
Nesterov, S.Y.
Netanel, N.W.
Netravali, R.A.
Neuenswander, K.
Neuhauser, D.
Neuman, R.D.
Newman, B.D.
Newman, S.D.
Newman, W.I.
Newton, J.
Neylon, J.
Ng, C.
Ng, L.C.
Ngo, C.
Nguyen, C.A.
Nguyen, D.K.
Nguyen, K.
Nguyen, S.T.
Nguyen, T.H.
Nguyen-Vo, T.
Nham, K.H.
Ni, C.K.
Ni, N.
Ni, P.
Nianogo, R.A.
Nickelsburg, G.
Niemann, C.
Niemi, C.A.
Nienhaus, J.
Nikbin, M.
Nikolov, L.A.
Nimmer, D.
Nishi, A.
Nishimoto, J.
Nishimura, I.
Niu, S.
Njabo, K.Y.
Noble, S.U.
Nogami, Y.
Noguera, P.A.
Noh, S.
Noji, J.M.
Nonacs, P.N.
Norberg, M.
Nordby, C.
Nordendale, N.
Noriega, C.A.
Norkey, A.J.
Normore, C.G.
North, M.A.
Northrop, G.S.
Nouri-Mahdavi, K.
Novales, N.V.
Novembre, J.P.
Novitch, B.G.
Nowatzki, A.J.
Nowicki, T.
Nuechterlein, K.H.
Nunan, T.A.
Nunez, J.
Nunez, R.L.
Nussbaum, F.A.
Nwankwo, E.
Nye, M.
O'Brien, E.
O'Connell, D.P.
O'Connor, T.K.
O'Dell, T.J.
O'Donnell, M.F.
O'Farrill, A.
O'Grady, C.
O'Neal, A.P.
O'Neill, B.
O'Neill, K.
O'Shea, J.M.
O'Sullivan, T.E.
Oak, S.
Oba, H.M.
Obara, I.
Obidike, O.J.
Ochoa, A.M.
Ochoa, A.R.
Ochs, E.
Oda, T.
Odeh, J.O.
Odinga, S.O.
Odland, T.
Ogawa, T.
Oh, J.
Oh, J.S.
Ohanian, D.
Ohanian, L.
Ohanian, V.A.
Ohemeng, A.O.
Okah, R.O.
Okimura, L.C.
Okin, G.S.
Oktay, N.
Okuhara, C.
Olabinjo, I.
Olcese, R.
Olde Loohuis, L.M.
Olivares Pesante,
Olivas, D.L.
Olivieri, J.M.
Olsen, F.E.
Olson, A.K.
Omwami, E.S.
Ong, M.K.
Ong, P.M.
Ong, R.A.
Ophoff, R.A.
Opie, C.S.
Opie, M.P.
Opp, M.
Orange, T.
Ordway, E.
Orellana, M.E.
Orenstein, A.A.
Orfield, G.A.
Orren, G.R.
Orren, K.J.
Orso, K.A.
Orsulic, S.
Ortiz, V.
Ortner, S.B.
Osborn, K.P.
Osborne, A.E.
Osei Twumasi, O.I.
Osher, S.J.
Oshiro, T.
Osipov, A.
Osman, M.
Osman, T.
Ostrovsky, R.
Oswald, S.M.
Ottolia, M.
Ouchi, W.G.
Outbakat, M.
Ow, F.P.
Oxford, Z.S.
Oyetimein, M.
Ozcan, A.
Ozel, G.
Ozimac, K.M.
Ozler, S.
Pack, E.R.
Packard, R.R.
Padilla Monroy, D.
Pagden, A.R.
Page, G.B.
Paige, D.A.
Paik Schoenberg, F
Paik, S.
Pajonk, F.
Pajukanta, P.E.
Pak, I.
Palencia, H.
Palmer, C.
Palmisano, C.J.
Palsberg, J.
Pamarti, S.
Pan, P.J.
Pan, Y.
Panageas, S.
Panagia, D.
Pandey, R.
Pandol, S.J.
Pang, J.
Pao, P.S.
Papadopoulos, J.K.
Papazian, D.M.
Papenburg, H.
Papp, J.C.
Paquette-Smith, M.
Parachini, N.P.
Parasher, T.
Paredes, O.
Paredes, V.A.
Parhami, F.
Park, A.
Park, C.
Park, H.
Park, J.
Park, J.B.
Park, J.J.
Park, K.
Park, K.R.
Park, N.
Park, P.S.
Park, S.
Park, S.J.
Parkinson, C.M.
Parks, S.A.
Parodi Ambel, K.
Parson, E.
Partin, C.
Pasaniuc, B.
Pascal, U.P.
Passos, J.L.
Pasternak, N.J.
Pat-Osagie, I.
Patchigondla, J.
Patel, N.
Patel, S.
Patel, V.
Patino Loira, J.
Patt, S.S.
Patterson, R.
Pau, G.
Paul, K.N.
Paul, T.
Paull, M.
Paulson, S.E.
Pavlish, C.L.
Payne, G.S.
Payne, J.K.
Payson, J.
Pearl, J.
Pearlstein, E.J.
Pebley, A.R.
Peccei, R.
Pederson, D.J.
Pei, Q.
Pelaez, A.R.
Peleg, E.
Pelikhova, J.
Pellegrini, C.
Pellegrini, M.
Pelliccioni, L.S.
Pelligra, E.
Pelowich, K.A.
Peltonen, L.
Peltzer, G.F.
Pena, C.L.
Peng, C.
Peng, N.
Peng, S.
Peng, Y.
Penny, H.
Perez Gonzalez, I.
Perez Truglia, R.N
Perez, E.O.
Perez, F.A.
Perez, G.
Perez, J.
Perez-Montesinos,
Perez-Torres, R.
Peris, A.
Peris, T.S.
Perkins, L.L.
Perlman, H.
Perluss, D.
Pernas, L.
Perrigo, J.L.
Perry, J.
Perry, K.
Perry, S.E.
Persaud, D.L.
Pescador, O.
Pestana, C.G.
Peters, M.E.
Petersen, P.
Peterson, J.D.
Peterson, M.A.
Peterson, R.
Peterson, S.
Petiaskina, M.A.
Petigura, E.A.
Petta, J.R.
Pettibone, W.D.
Pfluegl, G.M.
Pham, H.D.
Pham, H.V.
Phelan, J.P.
Phelps, B.J.
Phelps, L.L.
Phelps, P.E.
Philip, T.M.
Phillips, A.
Phillips, D.D.
Phillips, L.R.
Phillips, M.
Piacentini, J.C.
Piano, N.
Pierce, G.S.
Pierce, M.D.
Pierre, J.
Pierson, T.M.
Pieters, H.C.
Pike, N.A.
Pike, S.W.
Pilgram, J.J.
Pilon, L.G.
Pilshchikov, I.
Pimentel, H.J.
Pimentel, M.
Pina, N.
Pincetl, S.S.
Ping, P.
Pinkney, R.
Pinney, G.D.
Pinter-Wollman, N.
Pioustin, G.
Piper, W.S.
Pires, D.B.
Pirih, F.Q.
Pirino, G.
Pisegna, J.R.
Pitz, E.
Placek, K.
Plann, S.J.
Plath, K.
Pledger, J.M.
Poe, G.R.
Pogossian, M.
Poliquin, C.W.
Pollack, G.J.
Ponce, N.A.
Poole, E.J.
Popa, S.
Popham, S.A.
Popkin, L.A.
Popoli, J.R.
Popovic, D.
Porciani, N.
Porter, M.A.
Porter, T.M.
Portera-Cailliau,
Posner, D.N.
Posner, M.
Post, J.F.
Potkonjak, M.
Potthoff, S.J.
Pottie, G.J.
Potts, S.S.
Pourat, N.
Pouratian, N.
Pourzangi, B.
Powers, E.M.
Pozuelo, M.
Prabhu, A.
Prado, D.
Prak, R.
Prasanna, S.
Prelip, M.L.
Prescott-Johnson,
Presner, T.S.
Presson, A.P.
Price, R.
Prikhodko, S.
Prins, R.M.
Priselac, J.Z.
Privett, C.A.
Provenzano, C.A.
Prunet, N.
Psaras, C.
Pu, C.
Puig Saus, C.
Purdy, W.C.
Purnanandam, A.
Purves, A.C.
Putterman, S.J.
Pyle, A.D.
Qasawadish, H.
Qi, X.
Qian, Y.
Qiao, Y.
Qin, L.
Qiu, R.
Qu, Z.
Quadlin, N.M.
Quang, C.T.
Quartz, K.
Quddus, S.
Que Hee, S.S.
Quicoli, A.C.
Quijada, P.J.
Quijano, V.
Quinlan, M.E.
Quinones, P.
Qunell, S.D.
Qutami, L.N.
Rabin, N.A.
Radd, B.
Radinsky, A.
Radu, C.G.
Radu, R.
Rafey, W.M.
Ragadio, J.
Raghavan, R.
Ragir, J.W.
Rahmat-Samii, Y.
Raia, F.
Rajabally, T.C.
Rajagopal, D.
Rajapakse, A.C.
Rajaram, K.
Rajaraman, S.
Ralli, A.
Raman, A.P.
Ramdeen, A.
Ramezani, R.
Ramirez Valencia,
Ramirez, C.M.
Ramirez, G.
Ramirez, I.M.
Ramirez-Villatoro,
Ramos, C.
Ramos, C.L.
Rana, K.
Rando, T.A.
Ranero Echeverria,
Ranz, J.H.
Rao, D.S.
Rao, J.Y.
Raphael, M.N.
Rashidi, J.S.
Rastegar, R.Z.
Raustiala, K.
Raval, N.A.
Ravetch, D.S.
Ravetto, K.S.
Rawls, W.J.
Ray, L.A.
Raynor, O.U.
Razack, S.H.
Razavi, B.
Razinia, Z.
Re, L.
Reas, C.
Reda Coll, F.
Reddy, S.T.
Redmond, S.L.
Reed, E.F.
Reed, T.A.
Reeder, K.B.
Rees, H.M.
Reese, M.A.
Refuerzo, B.J.
Regan, B.C.
Rehan, V.K.
Reich, P.L.
Reichman, R.
Reid, C.
Reifel, N.M.
Reigle, R.F.
Reiher, P.L.
Reilly, A.
Reiner, E.S.
Reinhard, K.
Reinman, G.D.
Reischl, H.H.
Reise, S.P.
Reisler, E.
Reizman, L.
Ren, Y.
Rene, R.W.
Rensel, M.A.
Repetti, R.L.
Rescorla, M.A.
Reslan, R.
Rett, J.L.
Reue, K.
Rexach, J.
Reyes Noriega, S.
Reynolds, H.M.
Rezk-Hanna, M.
Rhie, K.K.
Rhodd-Lee, H.M.
Ribas, A.
Ribeiro Antunes Pi
Ribet, E.R.
Rice, L.N.
Rice, T.H.
Rich, M.
Richardson, N.
Richlin, A.E.
Richman, W.M.
Rigby, D.
Rigg, R.
Rigueur, D.
Riley, A.R.
Rimal, P.
Rimoin, A.W.
Ringach, D.L.
Rios Aguilar, C.
Rissman, J.A.
Rith-Najarian, L.R
Ritter, Z.S.
Ritterbusch, A.E.
Ritz, B.R.
Rivera-Murillo, R.
Robbins, T.
Robbins, W.
Robbins, W.A.
Roberge, H.L.
Roberson, K.B.
Roberts, A.F.
Roberts, C.
Roberts, I.
Roberts, M.T.
Roberts, S.T.
Roberts, T.
Robertson, T.M.
Robiglio, A.
Robin, A.
Robinson-Sweet, A.
Robles, T.F.
Roca, R.E.
Roca, V.
Rocco, R.A.
Roces, B.
Rocheadachi, S.
Roddy, K.P.
Rodilitz, S.R.
Rodrigues Serra Si
Rodrigues Sobreira
Rodriguez Drissi,
Rodriguez, G.T.
Rodriguez, J.A.
Rodriguez, J.N.
Rodriguez, N.N.
Roels, G.Y.
Rogers, J.S.
Rogers, L.
Rogers, M.L.
Rogowski, R.L.
Rohanna, K.
Rojas, R.R.
Roll, R.W.
Rome, L.H.
Romero, R.
Romero, R.C.
Rompczyk, J.D.
Root, J.
Roper, M.L.
Rosario, R.R.
Rose, N.R.
Rosen, A.E.
Rosen, J.
Rosenberg, J.M.
Rosenblum, B.T.
Rosenboom, D.A.
Rosenfeld, J.L.
Rosenfeld, K.N.
Rosenstock, L.
Rosenthal, N.G.
Rosenzweig, J.
Rosner, F.
Ross, M.L.
Ross, R.S.
Ross, S.G.
Rossi, G.
Rossi, P.E.
Rossman, G.
Rostamian, F.D.
Roth, M.D.
Rothberg, M.
Rotter, J.I.
Rouquier, R.A.
Roush, S.
Rousseve, D.J.
Roverud, R.C.
Rowat, A.C.
Rowe, B.T.
Rowe, K.E.
Roy, A.
Roy, R.
Roychowdhury, V.P.
Rozenblatt, D.C.
Rozengurt, J.E.
Rshtouni, M.
Ruan, D.
Rubens, M.A.
Rubin, Y.F.
Rubinson, G.J.
Rude, E.D.
Rudnick, J.A.
Rudraraju, P.U.
Rueter, M.O.
Rui, D.K.
Ruiz, M.
Ruiz, O.
Ruiz, T.F.
Rulliere, C.P.
Runnegar, B.N.
Runyan, M.T.
Ruperto, M.F.
Rushing, R.A.
Russ, B.D.
Russell, A.A.
Russell, C.T.
Russell, D.J.
Russell, M.A.
Saavedra Lux, D.
Sabol, T.A.
Sabrkhani, N.S.
Saby, A.H.
Sack, L.
Sackett, A.R.
Sadeghi Hosseini,
Sadzik, T.M.
Sagasti, A.
Sager, L.G.
Saguy, A.C.
Saha, A.
Sahai, A.
Saib, S.M.
Said, J.W.
Saide Peralta, P.
Saint Martin Guerr
Sakai, S.
Sakakibara, M.
Salah, D.
Salazar, P.
Salazar, R.
Salem, B.E.
Sallam, T.I.
Salmon, J.
Salmon, M.R.
Saltzberg, D.
Salzman, J.E.
Samani, J.F.
Sambrano, S.L.
Sampath, A.P.
Samuel-Nakamura, C
Samuelson, M.G.
San Juan, C.D.
Sanchez Ocal, P.
Sanchez, A.
Sanchez, J.L.
Sanchez, R.E.
Sanchez, W.S.
Sander, R.H.
Sanders, A.
Sanders, B.
Sanders, B.J.
Sandesara, U.N.
Sandhofer, C.M.
Sandmeier, N.S.
Sandoval Belmar, M
Sandoval, W.A.
Sangalang, C.C.
Sangiorgio, S.N.
Sankararaman, S.
Sant, G.
Santa Ana, O.
Santachiara, P.
Santana, T.L.
Santhanam, A.P.
Santiago Tierno, I
Santibanez, L.
Santikian, L.N.
Santos Da Silveira
Santos Trinidad, C
Santos, A.
Santos, C.E.
Santos, V.
Sarada, U.
Sarin, R.K.
Sarkar, S.
Sarkissyan, T.
Sarrafzadeh, M.
Saucedo, G.
Saunders, J.
Sautet, P.
Savage, R.W.
Savage, V.M.
Savoca, P.W.
Sawant, V.M.
Sawyer, M.Q.
Sax, L.J.
Sayeed, A.
Sayegh, P.
Saywitz, K.J.
Scafoglio, C.
Scallen, E.
Scalzo, F.
Scelza, B.A.
Scerri, E.R.
Schaberg, D.C.
Schachner, G.T.
Schaeffer, D.B.
Schaeffer, H.K.
Schagene, J.D.
Schaller, R.T.
Schauble, E.A.
Schaue, D.
Schein, S.J.
Scheinrock, J.I.
Schiestl, R.H.
Schiff, S.J.
Schilling, R.
Schlag, J.D.
Schlichting, H.E.
Schlinger, B.
Schloss Arrieta, A
Schmidt, J.J.
Schmidtke, H.
Schmitt, D.L.
Schmitt, M.A.
Schnaps, A.
Schniedewind, W.M.
Schopf, J.W.
Schorn, J.M.
Schriver, D.
Schubert, G.
Schulz, K.M.
Schumann, J.H.
Schutze, C.T.
Schwalbe Koda, D.
Schwartz, B.J.
Schwartz, E.S.
Schwartz, J.A.
Schwartz, J.C.
Schwartz, T.
Schwartz, T.E.
Schwarz, K.
Schweizer, F.E.
Schwieterman, N.T.
Scorza, D.J.
Scott, D.A.
Scott, E.
Scott, K.H.
Scroggins, M.J.
Scumpia, P.O.
Seamans, M.J.
Sears, D.O.
Sears, K.E.
Seet, C.S.
Segura, G.M.
Sehatbakhsh, N.
Sehl, M.E.
Seibt, U.
Seidlits, S.K.
Seiler, S.A.
Seki, E.
Selbst, A.D.
Sellars, P.M.
Seltzer, M.
Seltzer, M.H.
Senturk, D.
Sereesuchart, K.
Serrano, C.P.
Setoguchi, E.H.
Setrakian, N.
Seu, M.
Sevilhano Puglieri
Sew Hoy, A.M.
Sexton, J.S.
Shababi, L.I.
Shackelford, D.B.
Shadduck-Hernandez
Shaddy, F.P.
Shafaat, H.
Shaffer, B.
Shafir, S.C.
Shah, H.
Shah, P.K.
Shah, S.
Shahidzadeh, A.
Shaked, S.
Shamarova, A.
Shams, L.
Shamshoni, J.S.
Shanahan, C.
Shaner, A.
Shao, H.
Shapiro, N.E.
Shapley, A.E.
Share, J.
Sharif, B.
Sharif, M.
Sharifi, R.T.
Sharma, A.
Sharma, M.
Sharma, S.
Sharpe, J.
Sharpe, M.J.
Sharrah, R.K.
Sharvit, Y.
Shattuck, D.
Shaw, A.J.
Shayani, S.
Shayegan, M.R.
Sheetz, C.E.
Shem-Tov, Y.
Shen, A.
Shen, K.
Shen, L.
Shen, Y.
Sheng, K.
Sheng, S.
Sheng, Y.
Sheppard, D.
Sherman, M.
Sherman, R.
Sherrer, T.A.
Sherstov, A.
Sherwood, L.A.
Shetty, V.
Shevchenko, O.
Shi, K.W.
Shi, Q.
Shi, W.
Shideler, R.P.
Shiffrin, S.
Shih, M.J.
Shih, S.
Shimazaki, S.
Shin, B.Y.
Shin, D.D.
Shin, J.
Shin, K.
Shin, M.E.
Shinderman, M.
Shinnick, M.A.
Shintre, Y.
Shipe, R.F.
Shirihai, O.
Shivkumar, K.
Shlyakhtenko, D.Y.
Shneyder, V.
Shodahl, S.
Shook, M.N.
Shors, I.
Shorter, D.D.
Shorter, W.
Shoup, D.C.
Shrivastava, R.
Shu, S.B.
Shuai, K.
Shuger, D.K.
Shushtari, P.J.
Si, A.T.
Siddiq, A.A.
Sidhu, M.K.
Sidwell, A.
Sietsema, A.N.
Sigar, P.
Sigmon, J.P.
Silva, A.J.
Silva, C.S.
Silverman, D.H.
Silvers, J.A.
Silvestri, G.
Sim, M.
Simmons, C.M.
Simmons, D.D.
Simon, O.
Simon, P.A.
Simon, W.E.
Simonetti, D.A.
Simonian, R.S.
Simpson, A.N.
Simpson, M.E.
Sindhu, S.
Singh, R.R.
Sinha, V.
Sinsheimer, J.S.
Sisodiya, L.
Sissa, G.
Sivakumar, A.
Sklar, S.
Sladeck, J.J.
Slamon, D.J.
Slavich, G.M.
Sletten, E.M.
Sloan, H.E.
Slusser, W.M.
Slyomovics, S.
Smale, S.
Small, L.A.
Small-Rodriguez, D
Smallberg, D.A.
Smalley, K.
Smith, B.J.
Smith, C.
Smith, C.S.
Smith, D.
Smith, J.L.
Smith, L.E.
Smith, L.V.
Smith, M.F.
Smith, M.G.
Smith, M.L.
Smith, M.M.
Smith, S.R.
Smith, T.B.
Smits, T.J.
Smoak, J.D.
Smurda, J.D.
Snelson, D.S.
Snipes, J.
Soakai, S.L.
Soatto, S.
Sobel, E.M.
Sofroniew, M.V.
Sohn, S.S.
Sokolich, M.
Sokolow, S.
Soley Mateu, A.
Solon, M.P.
Solone, C.J.
Solorzano, D.
Song, Y.
Sonthalia, R.S.
Soo, C.
Sood, S.
Soper, D.C.
Soragni, A.
Sorenson, O.J.
Sork, V.
Soroka, S.N.
Soroosh, A.
Sorvillo, F.J.
Sotillo, L.
Soto Mejia, J.
Souly, W.G.
Souza, L.
Spagnola, N.A.
Spalding, L.A.
Spearrin, R.M.
Speed, S.E.
Speer, I.D.
Speier, W.F.
Spellman, P.T.
Spencer, L.R.
Spencer, M.J.
Speyer, J.L.
Spiegel, S.L.
Spielberg, L.M.
Spigelman, I.
Spillenger, C.S.
Spiller, S.
Splint, M.A.
Spokoyny, A.M.
Sportiche, D.L.
Spratford, C.M.
Spruill, M.J.
Srikanth, S.
Srinivasan, R.
Srivastava, M.B.
Srivastava, S.
St John, M.A.
Staba, R.J.
Stable, A.J.
Stacey, P.J.
Stahovich, K.
Stahuljak, Z.
Staley, D.A.
Stanish, C.S.
Stanton, A.L.
Stark, K.J.
Stark, T.E.
Staub, S.D.
Steen, F.F.
Stefanovska, M.
Stefans, B.K.
Stein, A.A.
Stein, D.R.
Stein, S.
Steinberg, M.G.
Steinert-Threlkeld
Steinmetz, J.A.
Stenstrom, M.K.
Stephan, M.
Stephenson, J.C.
Stern, E.I.
Stern, S.B.
Stern, S.C.
Sternheimer, S.
Sternini, C.
Stevens, C.M.
Stevenson, B.
Stewart, A.L.
Stewart, J.M.
Stewart, J.P.
Stigers, S.
Stigler, J.W.
Stivers, T.J.
Stixrude, L.P.
Stockton, R.
Stoll, M.A.
Stone, B.A.
Stone, B.D.
Stone, R.
Storey, C.
Storper, M.C.
Stowell, T.A.
Stoyanova, T.I.
Stratton, S.J.
Streeter, C.A.
Streit, D.
Stremitzer, A.
Strickland, L.M.
Stroessner, S.J.
Stubblefield, O.A.
Studer, K.M.
Stulberg, N.H.
Stumbaugh, D.L.
Sturgis, C.A.
Stutz, J.P.
Su, M.A.
Suarez-Orozco, C.E
Subotnik, K.L.
Subrahmanyam, A.
Subrahmanyam, S.
Subramaniam, L.M.
Suchard, M.A.
Suddath, R.L.
Sudhinaraset, M.
Suffet, I.H.
Sugar, C.A.
Suh, J.
Suh, N.
Suhr, C.
Suhr, J.W.
Sul, J.
Suman, M.
Summerhill, W.R.
Sumner, J.A.
Sun, H.
Sun, R.
Sun, X.
Sun, Y.
Sundara, M.
Sung, K.H.
Sung, W.
Suominen, K.
Supriya, K.
Surro, C.J.
Sussman, D.B.
Sussman, E.H.
Sutardja, R.
Suthana, N.A.
Sutherland, T.
Sutrisno, J.
Svendsen, R.D.
Sweeney, M.M.
Swenberg, J.H.
Swendeman, D.T.
Sykes, M.M.
Systla, V.
TA
Tabis, J.G.
Tabuada, P.
Tache, Y.
Taciroglu, E.
Tafari, M.A.
Taghavi, A.
Taira, K.
Taira, R.K.
Tajima, R.E.
Takahashi, L.M.
Takakura, A.H.
Takatsuka, Y.
Takeuchi, M.
Talison, J.
Talley, M.D.
Talton, L.
Tamai, L.Y.
Tamanoi, F.
Tamanoi, M.
Tambe, I.
Tamir, Y.
Tammineedi, A.
Tan, H.
Tan, Q.
Tan, Z.
Tandler, J.M.
Tang, C.S.
Tang, X.
Tang, Y.
Tanio, N.A.
Tanouye, J.
Tao, H.
Tao, T.C.
Tao, Y.
Tarling, E.J.
Tarnawska Senel, M
Taub, B.
Tausanovitch, C.N.
Tavrow, P.A.
Tawil, N.J.
Taylor, B.D.
Taylor, B.H.
Taylor, C.
Taylor, C.B.
Taylor, D.
Taylor, D.A.
Taylor, K.B.
Taylor, P.C.
Taylor, T.D.
Taylor, Y.
Teeter, W.G.
Teitell, M.A.
Tekin, D.
Telesca, D.
Tenn, S.E.
Teoh, S.
Teran, J.M.
Teranishi, R.
Terech, A.
Terraciano, K.B.
Terriquez, V.
Terzopoulos, D.
Teslaa, T.A.
Tetradis, S.
Thames, A.D.
Than, L.T.
Thaxton, S.
The Staff
Theantanoo, T.
Thies, M.F.
Thomas Tobin, C.
Thomas, A.
Thomas, D.R.
Thomas, E.A.
Thomas, M.A.
Thomas, M.R.
Thompson, D.
Thompson, R.M.
Thorngren, R.G.
Thornton, R.
Throop, C.J.
Thulin, L.
Tian, Y.
Tidball, J.G.
Tien, I.S.
Tienson-Tseng, H.L
Tilden, J.
Tillakaratne, N.J.
Tilly, C.C.
Timmerman, J.M.
Timmermans, S.
Tine, B.
Ting, K.
Tingley, M.W.
Tobin, A.J.
Tobin, S.C.
Tolbert, A.J.
Tolbert, S.H.
Tolentino, D.A.
Toma, J.
Tomboulis, E.T.
Tomita, N.
Tomiyama, J.
Tomlinson, M.L.
Tompkins, J.D.
Ton-That, H.
Tong, J.
Tontonoz, P.
Toohey, D.M.
Tornell, A.
Torrence, W.H.
Torrens, P.R.
Torres Franco, K.A
Torres Pacheco, S.
Torres, C.A.
Torres, I.M.
Torres, J.
Torres, J.J.
Torres-Gil, F.M.
Totaro, B.
Totten, G.G.
Tovey, C.E.
Trachtenberg, J.
Trammell, G.B.
Tramo, M.J.
Tran, J.
Tran, M.C.
Tran, S.
Travis, G.H.
Traweek, S.J.
Treisman, D.S.
Treu, T.L.
Treude, T.I.
Tricaud, C.
Trice, J.N.
Trimble, A.C.
Tripati, A.K.
Trivedi, S.
Trojan, C.
Trott, J.M.
Trueman, B.M.
Truong, H.T.
Truong, S.K.
Tsagkaropoulou, A.
Tsai, C.
Tsao, T.
Tseng, C.
Tseng, H.
Tserkovnyak, Y.
Tsiang, M.
Tsou, J.C.
Tsui, E.
Tsui, Y.
Tu, Z.
Tuazon, S.N.
Tudzarova-Trajkovs
Tung, N.C.
Tunstall, J.D.
Tuohy, D.
Turkson, S.
Turlo, K.A.
Turner, D.C.
Turner, J.L.
Turner, M.F.
Turner, P.A.
Turner, V.K.
Tutino, S.
Tward, D.J.
Tyler, D.
Tyler, M.J.
Tymochko, S.J.
Tyson, J.
Uddin, L.Q.
Uehara, W.F.
Uhm, S.
Uittenbogaart, C.
Ullmen, J.B.
Ulmert, H.S.
Ulrich, R.K.
Umar, S.
Umemoto, K.N.
Ungprateeb Flynn,
Unzueta, M.M.
Upchurch, D.M.
Upton, D.
Upton, E.R.
Uzal, A.
Vaidya, A.
Valentine, J.
Valentine, J.S.
Valenzuela, A.
Valenzuela, J.L.
Valenzuela, R.A.
Vallim, T.A.
Van Dam, M.
Van Delden, M.H.
Van Deloo, L.D.
Van den Broeck, G.
Van der Bliek, A.M
Van der Schaar, M.
Van Dyk, R.R.
Van Niel, J.P.
Van Rooy, E.E.
Van Schilfgaarde,
Van Valkenburgh, B
Van Veen, J.E.
Vancour, S.G.
Vandenberghe, L.
Vanderveer, B.J.
Vanderveer, E.R.
Vannucchi, D.
Varfolomeeva, P.
Varghese, G.
Varnado, S.
Varney, R.H.
Varvarezos, K.K.
Vaseghi, M.
Vashakmadze, S.
Vashchilenko, D.
Vasquez, S.S.
Vassiliev, V.V.
Vaughn, R.A.
Vavreck, L.
Vazquez, A.M.
Vazquez, J.I.
Vela, J.
Velez Ramirez, D.E
Velli, M.C.
Venkat, B.J.
Ventura, A.
Venugopal, S.
Verducci, P.M.
Vergara, J.L.
Vergara, X.P.
Vernon, M.
Verstein, A.
Verzemnieks, I.L.
Vese, L.A.
Vesna, V.
Vestal, M.A.
Vicaria Angel, M.
Vikram, A.
Villanueva, C.J.
Villarejo, A.
Villarreal, A.
Villarreal, E.E.
Villasenor, J.D.
Vine, B.H.
Vinters, H.V.
Virani, A.
Visan, M.
Vitello, M.C.
Vivian, A.L.
Vlashi, E.
Vo Dang, T.
Vogel, J.E.
Voigtlaender, N.
Volokh, E.
Von Blum, P.
Von Ehrenstein, O.
Von Falkenhausen,
Von Glahn, R.
Von Zur Muehlen, E
Vondriska, T.M.
Voskuhl, R.R.
Voss, M.
Vossel, K.A.
Votava, C.
Vriesman, L.J.
Vroon, R.W.
Vural, Z.
Vyas, N.N.
Wabote, A.T.
Wachs, M.
Wacziarg, R.T.
Wagar, P.M.
Waggoner, J.P.
Wagman, J.A.
Wagmister, C.
Wagner, C.A.
Wahbeh, J.M.
Wake, T.A.
Waldinger, R.
Walker, A.P.
Walker, D.W.
Walker, E.T.
Walker, R.J.
Wall, T.R.
Wallace, J.W.
Walsh, S.
Walske, J.M.
Wanebo, T.A.
Wang, A.
Wang, B.
Wang, C.
Wang, H.
Wang, J.
Wang, J.C.
Wang, J.J.
Wang, K.L.
Wang, L.S.
Wang, M.
Wang, N.
Wang, Q.
Wang, R.
Wang, S.
Wang, T.
Wang, W.
Wang, X.
Wang, Y.
Wang, Z.
Ward, J.M.
Ward, K.D.
Warlaumont, A.S.
Warri, V.R.
Warshauer, C.
Wartanian, R.J.
Waschek, J.A.
Wasdahl, A.J.
Washington, A.M.
Washington, M.
Wasserman, M.S.
Wasson, J.T.
Wassum, K.
Waterman, A.D.
Watkins, A.L.
Watson, D.C.
Watson, R.N.
Watts, C.E.
Waugh, J.
Wayne, R.
We, D.
Weaver, E.
Webb, N.M.
Weber, M.E.
Webster, N.D.
Wedgeworth, L.D.
Wegemer, C.
Weidhaas, J.B.
Weill, P.
Weinhaus, E.A.
Weinstein, M.D.
Weir, M.A.
Weisburst, E.K.
Weise, S.O.
Weiss, P.S.
Weiss, R.
Weiss, R.L.
Weiss, S.
Weissman, B.
Welch, I.I.
Wells, K.B.
Wells, M.
Wells, W.
Wells, W.R.
Weltman, G.
Weltman, S.
Wen, J.
Wen, P.J.
Wendrich, W.Z.
Wenten, I.N.
Wertlieb, N.J.
Wesel, R.D.
Wessels, D.W.
West, A.I.
Westerink, S.H.
Westphaln, K.K.
Westrup, L.A.
Wetanson, R.A.
Wetzstein, S.R.
Weyi, B.
Wharton, G.
Wheaton, B.N.
White, S.
White, S.A.
White, T.J.
White-Clayton, D.L
Whitehorn, N.
Whitfield, N.J.
Whiting, W.C.
Whitson, J.A.
Whittell, J.C.
Whyte, C.D.
Wickman, P.A.
Widess, K.A.
Wijaya, J.
Wikenheiser, A.M.
Wiley, D.J.
Wiley, L.F.
Wilke, L.A.
Wilkinson, D.O.
Williams, A.M.
Williams, B.S.
Williams, D.S.
Williams, E.C.
Williams, G.A.
Williams, J.
Williams, J.A.
Williams, L.A.
Williams, L.M.
Williams, M.G.
Williams, P.
Williams, S.S.
Willis, H.L.
Wilson, B.
Wilson, B.D.
Wilson, C.
Wilson, C.H.
Wilson, D.L.
Wilson, J.L.
Wilson, M.
Wilson, N.M.
Wilson, P.A.
Wilson, R.
Winkle, M.D.
Winkler, A.D.
Winn, D.
Winters, D.R.
Wirz, R.E.
Witte, O.N.
Wohlschlegel, J.A.
Wolff, G.
Woller, M.P.
Wollman, R.
Wolpe, S.M.
Wong, C.
Wong, D.A.
Wong, D.T.
Wong, G.C.
Wong, J.
Wong, N.D.
Wong, R.
Wong, R.B.
Wong, W.
Wong, Y.
Wonsowicz, P.
Woo, B.K.
Woo, E.A.
Woo, J.C.
Wood, B.M.
Wood, C.T.
Wood, J.J.
Wood, S.E.
Woods, R.P.
Woodworth, K.
Woolf, G.
Worger, W.H.
Woronoff, M.A.
Wortham, J.S.
Worthley, A.V.
Wray-Lake, L.
Wrench, L.
Wright, A.R.
Wright, E.L.
Wright, E.M.
Wright, T.A.
Wu, B.M.
Wu, C.
Wu, E.C.
Wu, G.
Wu, H.
Wu, H.H.
Wu, J.S.
Wu, J.Y.
Wu, J.Z.
Wu, L.
Wu, Q.
Wu, S.
Wu, T.
Wu, W.
Wu, Y.
Wuest, B.G.
Wurster, T.S.
Wyatt, G.E.
Wyatt, M.M.
Xiao, X.
Xie, Y.
Xin, C.
Xing, Y.
Xiong, G.
Xiong, H.
Xu, A.
Xu, C.
Xu, H.
Xu, P.
Xu, S.
Xuan, Z.
Xue, A.J.
Xue, Y.
Yadav, C.
Yan, Y.
Yanez, J.
Yang, C.K.
Yang, H.
Yang, J.
Yang, L.
Yang, O.O.
Yang, Q.
Yang, T.
Yang, X.
Yang, X.W.
Yang, Y.
Yaniv, Y.
Yano, E.M.
Yao, J.
Yao, V.
Yao, W.
Yao, Y.
Yarborough, R.A.
Yarbrough, L.B.
Yaroslavsky, Z.
Yates, A.
Yates, A.D.
Yates, L.T.
Yates, P.F.
Yavuz, M.
Ye, Z.
Yeates, T.O.
Yedid, A.J.
Yee, S.
Yee-Bradbury, C.M.
Yefimenkov, V.
Yeh, D.
Yeh, P.J.
Yeh, W.
Yelanyan, M.
Yenser, S.I.
Yeo, C.S.
Yeong, W.
Yeritsian, G.
Yeung, L.L.
Yi, S.
Yildiz, Y.
Yildiz, Z.
Yin, A.
Yin, J.
Yin, W.
Yin, W.E.
Yin, Y.
Ying, L.
Yirush, C.B.
Yokota, M.
Yokoyama, O.
Yonko, E.A.
Yoo, D.K.
Yoon, H.L.
Yoon, J.E.
Yoon, J.W.
Yoshida, M.
Young, A.
Young, A.S.
Young, D.E.
Young, E.D.
Young, K.T.
Young, S.G.
Yu, B.
Yu, F.
Yu, J.
Yu, L.W.
Yu, M.
Yu, T.Y.
Yu, W.
Yu, X.
Yuan, K.F.
Yuan, M.
Yuan, X.
Yum, J.
Zack, J.A.
Zaelke, E.W.
Zaidel, D.
Zaitlen, N.A.
Zaller, J.R.
Zambetti, L.
Zame, W.
Zamudio, J.R.
Zaniolo, C.
Zanontian, L.A.
Zapata, F.
Zarrillo, S.
Zatz, N.D.
Zavala, D.R.
Zdencanovic, B.
Zeithammer, R.
Zeleny, C.L.
Zender, M.
Zeng, J.
Zeng, Y.
Zeng, Z.
Zenios, S.
Zepeda, J.C.
Zes, D.A.
Zewde, N.B.
Zeytinoglu, N.
Zhang, C.
Zhang, D.
Zhang, J.
Zhang, L.
Zhang, M.
Zhang, Q.
Zhang, S.
Zhang, X.
Zhang, Y.
Zhang, Z.
Zhao, C.
Zheleznyak, A.
Zhen, A.
Zheng, J.J.
Zheng, X.
Zhong, X.
Zhou, B.
Zhou, H.
Zhou, J.
Zhou, M.
Zhou, Q.
Zhou, X.J.
Zhu, S.
Zhu, T.
Zhu, X.
Zhu, Y.
Ziadie, B.A.
Zieve, A.
Ziffren, K.
Zimmerman, F.J.
Zink, J.I.
Zipperstein, S.E.
Zipursky, S.L.
Zocchi, G.
Zohar, D.Y.
Zoller, J.A.
Zolt, E.M.
Zong, Z.
Zuckerman, B.M.
Zukoff, S.
Zuo, X.
Zuraw, K.R.
Zweig, E.A.
"""

In [ ]:
dummy_profs = dummy_profs.split('\n')

In [76]:

dummy_profs = dummy_profs[1:-1]

In [ ]:
print(dummy_profs[1:-1])

In [92]:
import random

for row in discussions_template_array:
    random_index = random.randint(0, len(dummy_profs) - 1)
    row[4] = dummy_profs[random_index]


In [ ]:
# Print the modified discussions_updated_array
for row in discussions_updated_array:
    print(row)

Now, all I need to have some classes completely filled out and ready to populate the database is to give each class a unique course_code.  This will look like the following:

2 letters representing the dept_code i.e. CS for COMP SCI
5 characters representing the course_num, i.e. 00001 for Introduction to Black Studies
2 characters representing the discussion_code i.e. 1C


In [95]:
for row in discussions_updated_array:
    # Extract relevant information from the row
    dept_code = row[1]
    course_num = row[2]
    discussion_code = row[5]
    
    # Construct the course code
    if dept_code == 'COM SCI':
        course_code = 'CS' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'AERO ST':
        course_code = 'AS' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'POLI SCI':
        course_code = 'PS' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'MATH':
        course_code = 'MT' + '0' * (5 - len(course_num)) + course_num + discussion_code
    elif dept_code == 'AF AMER':
        course_code = 'AA' + '0' * (5 - len(course_num)) + course_num + discussion_code
        pass

    
    # Update the first column of the row with the constructed course code
    row[0] = course_code


In [ ]:
for row in discussions_updated_array:
    print(row)

below are failed attempts to form regular expressions that capture majors so that I can associate them with courses. 

In [48]:
unique_first_column_values = {}  # Step 1: Create an empty dictionary

unique_rows = []  # Initialize a list to store unique rows

# Step 2: Iterate over the array of rows
for row in discussions_updated_array:
    # Step 3: Check if the value in the first column already exists in the dictionary
    if row[0] not in unique_first_column_values:
        # Step 4: If the value doesn't exist, add it to the dictionary and keep the row
        unique_first_column_values[row[0]] = True
        unique_rows.append(row)

# Step 5: After processing all rows, the dictionary will contain unique values from the first column.
# Step 6: Reconstruct the array with only the unique rows.
final_courses_array = unique_rows

In [ ]:
for row in final_courses_array:
    print(row)

In [ ]:
for row_index, row_data in enumerate(discussions_updated_array[:120], start=1):
    print(f"Row {row_index}: {row_data}")

In [51]:
final_courses_array = discussions_updated_array

In [ ]:
for row_index, row_data in enumerate(final_courses_array[:12], start=1):
    print(f"Row {row_index}: {row_data}")

In [97]:
test_courses_array = final_courses_array[50:100]

In [ ]:
test_courses_array

Here I will attempt to insert a few classes into our database before I populate it completely to avoid crashing the server

In [ ]:
pip install psycopg2-binary

In [98]:
import psycopg2
#replace the user, password, etc with actual data
connection = psycopg2.connect(user="USER", password="PSWD", host="HOST", port="PORT", database="DATABASE")
connection.autocommit = True
with connection.cursor() as cur: 
    for row in test_courses_array:
        course_code, dept_code, course_num, course_name, prof, discussion_code = row
        cur.execute (f"""
        INSERT INTO classes VALUES ('{course_code}', '{dept_code}', '{course_num}', '{course_name}', '{prof}', '{discussion_code}');""")
connection.close()

